## Proyek Analisis Data: E-Commerce Public Dataset
- **Nama:** Muhammad Ali Umar
- **Email:** al.emailkerja@gmail.com
- **ID Dicoding:** muhammadaliumar

## Menentukan Pertanyaan Bisnis

1. Bagaimana performa penjualan *e-commerce* berdasarkan lokasi geografis ?
2. Bagaimana persebaran konsumen dan nilai transaksi di berbagai wilayah ?

## Import Semua Packages/Library yang Digunakan

In [1]:
# library manipulasi data
import pandas as pd
import numpy as np
import math

# baca dataset
import kagglehub
import json
from urllib.request import urlopen

# mapping
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import plotly.graph_objs as go
import plotly.io as pio

# library visualisasi
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# pembelajaran mesin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import SplineTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# menyembunyikan peringatan agar output lebih bersih
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 500) # Untuk menampilkan semua kolom dari dataframe
pd.set_option("max_colwidth", None) # Untuk mengatur lebar kolom ke maksimum

## Data Wrangling

### Gathering Data

In [5]:
# mengambil dataset dari kaggle
olistbr_brazilian_ecommerce_path = kagglehub.dataset_download('olistbr/brazilian-ecommerce')

print('Impor sumber data selesai.')

Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Impor sumber data selesai.


In [6]:
# memuat dataset
df_customers = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv")
df_geolocation = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv")
df_order_items = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv")
df_order_payments = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv")
df_order_reviews = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv")
df_orders = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv")
df_products = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv")
df_sellers = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv")
df_product_category_name_translation = pd.read_csv("/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv")

### Assessing Data

In [7]:
def assess_dataframe(df, name):
    print(f"\n--- Assessing DataFrame: {name} ---")
    print(f"Shape: {df.shape}")
    print("Info:")
    df.info()
    print("\nNumber of missing values:")
    print(df.isnull().sum())
    print("\nDescriptive Statistics:")
    print(df.describe(include='all'))

# daftar dataframes untuk assess
dataframes = {
    "df_customers": df_customers,
    "df_geolocation": df_geolocation,
    "df_order_items": df_order_items,
    "df_order_payments": df_order_payments,
    "df_order_reviews": df_order_reviews,
    "df_orders": df_orders,
    "df_products": df_products,
    "df_sellers": df_sellers,
    "df_product_category_name_translation": df_product_category_name_translation
}

# Assess setiap dataframe
for name, df in dataframes.items():
    assess_dataframe(df, name)


--- Assessing DataFrame: df_customers ---
Shape: (99441, 5)
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB

Number of missing values:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Descriptive Statistics:
                             customer_id                customer_unique_id  \
count                              99441                             99441   
unique                    

**Insight:**

- Banyak *dataframe* yang tidak memiliki *missing value*.

- Namun, terdapat beberapa temuan penting yang perlu diperhatikan untuk proses *cleaning*, antara lain:

  - `df_order_reviews` memiliki *missing value* yang cukup banyak pada kolom `comment_title` dan `comment_message`.

  - `df_orders` memiliki missing value pada kolom `order_approved_at`, `order_delivered_carrier_date`,= dan `order_delivered_customer_date`.

  - `df_products` memiliki *missing value* pada kolom `category_name`, `name_length`, `description_length`, `photos_qty`, serta beberapa dimensi fisik produk.

- Selain itu, beberapa kolom jenis tanggal/*timestamp* pada `df_order_items`, `df_order_reviews` dan `df_orders` saat ini masih tersimpan sebagai *object* (*string*) dan perlu dikonversi ke tipe *datetime* agar dapat dianalisis secara benar.

### Cleaning Data

In [8]:
# mengonversi kolom ke tipe datetime
df_order_items['shipping_limit_date'] = pd.to_datetime(df_order_items['shipping_limit_date'])
df_order_reviews['review_creation_date'] = pd.to_datetime(df_order_reviews['review_creation_date'])
df_order_reviews['review_answer_timestamp'] = pd.to_datetime(df_order_reviews['review_answer_timestamp'])
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])
df_orders['order_approved_at'] = pd.to_datetime(df_orders['order_approved_at'])
df_orders['order_delivered_carrier_date'] = pd.to_datetime(df_orders['order_delivered_carrier_date'])
df_orders['order_delivered_customer_date'] = pd.to_datetime(df_orders['order_delivered_customer_date'])
df_orders['order_estimated_delivery_date'] = pd.to_datetime(df_orders['order_estimated_delivery_date'])

print("df_order_items info setelah konversi:")
df_order_items.info()
print("\n df_order_reviews info setelah konversi:")
df_order_reviews.info()
print("\n df_orders info setelah konversi:")
df_orders.info()

df_order_items info setelah konversi:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB

 df_order_reviews info setelah konversi:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------

In [9]:
# membuat kolom total_value
df_order_items["total_value"] = df_order_items["price"] + df_order_items["freight_value"]

df_order_items_ = df_order_items.groupby(by=["order_id", "product_id", "shipping_limit_date"]).agg(
    product_counts = ("product_id", "count"),
    total_price = ("price", "sum"),
    total_value = ("total_value", "sum")
).reset_index()
df_order_items_.head()

,order_id,product_id,shipping_limit_date,product_counts,total_price,total_value
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,2017-09-19 09:45:35,1,58.90,72.19
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,2017-05-03 11:05:13,1,239.90,259.83
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,2018-01-18 14:48:30,1,199.00,216.87
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,2018-08-15 10:10:18,1,12.99,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,2017-02-13 13:57:51,1,199.90,218.04


#### Menangani *missing value* pada `df_order_reviews`

In [10]:
df_order_reviews['review_comment_title'].fillna('no comment', inplace=True)
df_order_reviews['review_comment_message'].fillna('no comment', inplace=True)

print("Jumlah nilai yang hilang dsetelah mengisi 'no comment':")
print(df_order_reviews.isnull().sum())

Jumlah nilai yang hilang dsetelah mengisi 'no comment':
review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
dtype: int64


#### Menangani *missing value* pada `df_products`

1. Mengisi semua nilai yang hilang (NaN) pada kolom `product_category_name` di DataFrame `df_products` dengan string 'unknown'. Pastikan perubahan dilakukan secara langsung pada DataFrame (inplace).
2. Mengidentifikasi kolom numerik yang memiliki missing values di `df_products` (`product_name_lenght`, `product_description_lenght`, `product_photos_qty`, `product_weight_g`, `product_length_cm`, `product_height_cm`, `product_width_cm`).
3. Setiap kolom numerik yang teridentifikasi, mengisi nilai yang hilang (NaN) dengan nilai median dari kolom tersebut. Memastikan perubahan dilakukan secara langsung pada DataFrame (inplace).
4. Menampilkan jumlah nilai yang hilang untuk setiap kolom di `df_products` menggunakan `.isnull().sum()` untuk memverifikasi bahwa missing values telah ditangani.

In [11]:
df_products['product_category_name'].fillna('unknown', inplace=True)

numerical_cols_to_fill_products = [
    'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
    'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'
]

for col in numerical_cols_to_fill_products:
    df_products[col].fillna(df_products[col].median(), inplace=True)

print("Jumlah nilai yang hilang setelah pengisian:")
print(df_products.isnull().sum())

Jumlah nilai yang hilang setelah pengisian:
product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
dtype: int64


#### Menangani *missing value* pada `df_orders`

1. Mengisi nilai yang hilang (NaN) pada kolom `order_approved_at` di DataFrame `df_orders` dengan nilai dari kolom `order_purchase_timestamp`. Memastikan perubahan dilakukan secara langsung pada DataFrame (inplace).
2. Mengisi nilai yang hilang (NaN) pada kolom `order_delivered_carrier_date` di DataFrame `df_orders` dengan nilai dari kolom `order_approved_at`. Memastikan perubahan dilakukan secara langsung pada DataFrame (inplace).
3. Mengisi nilai yang hilang (NaN) pada kolom `order_delivered_customer_date` di DataFrame `df_orders` dengan nilai dari kolom `order_delivered_carrier_date`. Memastikan perubahan dilakukan secara langsung pada DataFrame (inplace).
4. Menampilkan jumlah nilai yang hilang untuk setiap kolom di `df_orders` menggunakan `.isnull().sum()` untuk memverifikasi bahwa missing values telah ditangani.

In [12]:
df_orders['order_approved_at'].fillna(df_orders['order_purchase_timestamp'], inplace=True)
df_orders['order_delivered_carrier_date'].fillna(df_orders['order_approved_at'], inplace=True)
df_orders['order_delivered_customer_date'].fillna(df_orders['order_delivered_carrier_date'], inplace=True)

print("Jumlah nilai yang hilang setelah pengisian:")
print(df_orders.isnull().sum())

Jumlah nilai yang hilang setelah pengisian:
order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64


**Insight:**
- Data yang telah dibersihkan dan diproses sebelumnya kini siap untuk analisis data eksploratori (EDA), pemodelan statistik atau tugas pembelajaran mesin, karena data tersebut mengatasi masalah kualitas data umum seperti kesalahan ketik dan nilai yang hilang.

- Analisis lebih lanjut dapat mencakup eksplorasi tren berbasis waktu dari kolom `datetime` yang baru dikonversi atau menyelidiki dampak kategori '*no comment*' dan '*unknown*' terhadap sentimen ulasan atau kinerja produk.

## Exploratory Data Analysis (EDA), Visualization & Explanatory Analysis

### Analisis Pelanggan


#### Pengeluaran Pelanggan Berdasarkan Geolokasi

In [13]:
# 1. Mengurutkan DataFrame `df_geolocation` berdasarkan kolom `geolocation_zip_code_prefix` secara ascending.
# 2. Mengelompokkan data berdasarkan `geolocation_city` dan `geolocation_state`.
# 3. Menyimpan hanya satu baris pertama dari tiap grup setelah data diurutkan.

data_geo = (
    df_geolocation
        .sort_values(by="geolocation_zip_code_prefix", ascending=True)  # Mengurutkan data
        .groupby(by=["geolocation_city", "geolocation_state"])          # Mengelompokkan data
        .head(1)                                                        # Mengambil baris pertama tiap grup
)

data_geo.head()


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
771,1001,-23.550498,-46.634338,sao paulo,SP
575,1001,-23.549779,-46.633957,são paulo,SP
10166,1307,-23.556812,-46.657135,sao bernardo do campo,SP
22261,2116,-23.515978,-46.582170,são paulo,RN
50999,3203,-23.216648,-46.861371,jundiaí,SP


In [14]:
# Menggabungkan data orders dengan order items
# Menyaring hanya pesanan yang berstatus delivered
# Menggabungkan dengan data pelanggan
# Mengelompokkan berdasarkan kota dan provinsi pelanggan
# Menghitung jumlah pesanan dan total nilai transaksi
# Mengurutkan berdasarkan jumlah pesanan (descending)
# Menggabungkan dengan data geolokasi yang mewakili tiap kota-provinsi

data_customers = (
    df_orders.merge(df_order_items, how="inner", on="order_id")               # Menggabungkan orders & order items
        .query('order_status == "delivered"')                                # Menyaring pesanan delivered
        .merge(df_customers, how="inner", on="customer_id")                  # Menggabungkan dengan data pelanggan
        .groupby(by=["customer_city", "customer_state"])                     # Group berdasarkan kota & provinsi
        .agg(
            orders_count=("order_id", "nunique"),                            # Jumlah pesanan unik
            total_value=("total_value", "sum")                               # Total nilai transaksi
        )
        .sort_values(by="orders_count", ascending=False)                     # Urutkan berdasarkan jumlah pesanan
        .merge(
            data_geo,
            how="inner",
            left_on=["customer_city", "customer_state"],
            right_on=["geolocation_city", "geolocation_state"]               # Gabung dengan data geolokasi
        )
)

data_customers["location"] = (
    data_customers["geolocation_city"].apply(lambda x: str(x).title())
    + ", "
    + data_customers["geolocation_state"]
)

data_customers.head(10)


,orders_count,total_value,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,location
0,15045,2107960.17,1001,-23.550498,-46.634338,sao paulo,SP,"Sao Paulo, SP"
1,6601,1111732.21,20010,-22.904775,-43.172688,rio de janeiro,RJ,"Rio De Janeiro, RJ"
2,2697,405950.51,30001,-19.923989,-43.935706,belo horizonte,MG,"Belo Horizonte, MG"
3,2071,345199.05,70002,-15.790439,-47.880655,brasilia,DF,"Brasilia, DF"
4,1489,238459.72,80002,-25.439563,-49.261963,curitiba,PR,"Curitiba, PR"
5,1406,209002.90,13010,-22.905552,-47.049448,campinas,SP,"Campinas, SP"
6,1342,214805.84,90001,-30.028364,-51.230589,porto alegre,RS,"Porto Alegre, RS"
7,1188,207713.30,40010,-12.969912,-38.511830,salvador,BA,"Salvador, BA"
8,1144,157735.65,7010,-23.467578,-46.529161,guarulhos,SP,"Guarulhos, SP"
9,911,116784.58,1307,-23.556812,-46.657135,sao bernardo do campo,SP,"Sao Bernardo Do Campo, SP"


#### Top 10 Penjualan Berdasarkan Geolokasi

In [16]:
# 1) Menginstal plotly (opsional, dilakukan satu kali)
!pip install -q plotly

# 2) Mengatur renderer
pio.renderers.default = "colab"   # penting saat digunakan di Google Colab

# 3) Mengecek isi dan ukuran data
print("Shape:", data_customers.shape)
print(data_customers[["location", "orders_count", "total_value"]].head(10))

# 4) Membuat visualisasi (pastikan data tersedia)
top10 = data_customers.head(10).copy()
max_orders = top10["orders_count"].max() if not top10["orders_count"].isna().all() else 0

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=top10["location"],
        y=top10["orders_count"],
        name="Jumlah Pesanan",                     # Jumlah pesanan
        marker_color="rgb(64,224,208)",
        hovertemplate="%{x}<br>Jumlah Pesanan: %{y}"
    )
)

fig.add_trace(
    go.Scatter(
        x=top10["location"],
        y=top10["total_value"],
        name="GMV",                                   # Nilai total transaksi (GMV)
        yaxis="y2",
        marker_color="rgb(255,160,122)",
        mode="lines+markers",
    )
)

fig.update_layout(
    title=dict(
        text="<b>Top 10 Penjualan Berdasarkan Geolokasi<b>",
        font=dict(size=12, family="Arial", color="black")
    ),
    plot_bgcolor="white",
    yaxis=dict(
        side="left",
        range=[0, max_orders + 1000],
        showgrid=False,
        zeroline=True,
        showline=False,
        showticklabels=True
    ),
    yaxis2=dict(
        side="right",
        overlaying="y",
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True
    ),
    xaxis=dict(showline=True, linecolor="rgb(204, 204, 204)", linewidth=2),
    legend=dict(orientation="h", x=0.8, y=1.1),
    annotations=[
        dict(
            text="Created By AL.",
            xref="paper", yref="paper",
            x=1, y=-.35,
            showarrow=False,
            font=dict(size=10, color="gray", family="Arial")
        )
    ],
    hovermode="x unified"
)

fig.show()  # gunakan renderer default (sudah disetel ke "colab")


Shape: (4214, 8)
                    location  orders_count  total_value
0              Sao Paulo, SP         15045   2107960.17
1         Rio De Janeiro, RJ          6601   1111732.21
2         Belo Horizonte, MG          2697    405950.51
3               Brasilia, DF          2071    345199.05
4               Curitiba, PR          1489    238459.72
5               Campinas, SP          1406    209002.90
6           Porto Alegre, RS          1342    214805.84
7               Salvador, BA          1188    207713.30
8              Guarulhos, SP          1144    157735.65
9  Sao Bernardo Do Campo, SP           911    116784.58


**Insight:**

- **São Paulo, SP** memimpin dalam hal volume pesanan dan GMV, menjadikannya sebagai pasar utama.
- **Rio de Janeiro** dan **Belo Horizonte** menunjukkan kinerja penjualan dan kontribusi pendapatan yang kuat.
- **Brasília, DF** memiliki lebih sedikit pesanan tetapi GMV tinggi, menunjukkan nilai pesanan rata-rata yang lebih tinggi.
- Variasi antara jumlah pesanan dan GMV di berbagai kota mencerminkan perilaku konsumen yang berbeda.

#### Pengeluaran pelanggan berdasarkan wilayah

In [17]:
# Mendefinisikan fungsi untuk melakukan Min-Max scaling pada satu nilai
# x: nilai yang akan diskalakan
# data: DataFrame yang berisi kolom terkait
# col: nama kolom yang akan diskalakan
def scaling(x, data, col):
    _min = min(data[col])  # mencari nilai minimum pada kolom
    _max = max(data[col])  # mencari nilai maksimum pada kolom
    return (x - _min)/(_max - _min)  # menerapkan rumus Min-Max scaling

# Menerapkan scaling pada kolom "orders_count"
# Membuat kolom baru "orders_count_scaling" berisi nilai yang telah diskalakan
data_customers["orders_count_scaling"] = data_customers["orders_count"].apply(
    lambda x: scaling(x, data_customers, "orders_count")
)

# Menerapkan scaling pada kolom "total_value"
# Membuat kolom baru "total_value_scaling" berisi nilai yang telah diskalakan
data_customers["total_value_scaling"] = data_customers["total_value"].apply(
    lambda x: scaling(x, data_customers, "total_value")
)

# Menghitung titik tengah geografis seluruh pelanggan
# lat_center: nilai rata-rata latitude
lat_center = np.mean(data_customers["geolocation_lat"])

# lon_center: nilai rata-rata longitude
lon_center = np.mean(data_customers["geolocation_lng"])

In [18]:
# Masukkan token Mapbox Anda di sini
token = "pk.eyJ1IjoiZGFvbWluaHRodWFuMDkxMjk2IiwiYSI6ImNtaGt1anc1cDFnOG0ybHI0N3BpemFjNTUifQ.nQFwkI3TpYfuqLoRonNfqA"

# Sesuaikan scaling
data_customers.loc[0:1, "total_value_scaling"] = data_customers.loc[0:1, "total_value_scaling"] / 2

fig = go.Figure(go.Scattermapbox(
        lat = data_customers["geolocation_lat"],
        lon = data_customers["geolocation_lng"],
        mode = "markers",
        marker = go.scattermapbox.Marker(
            size = data_customers["total_value_scaling"] * 300,
            sizemin = (data_customers["total_value"] / 12).min(),
            color = data_customers["orders_count_scaling"] * 100,
            cmin = (data_customers["orders_count"].min()) * 100,
            colorscale = "Teal"
        ),
        hoverinfo="text",
        hovertext=(
            data_customers["location"].astype(str) + "<br>" +
            "Number of orders: " + data_customers["orders_count"].astype(str) + "<br>" +
            "GMV: $" + data_customers["total_value"].round(2).astype(str)
        )
    )
)

fig.update_layout(
    autosize=True,
    margin={"r":5,"t":5,"l":5,"b":5},
    hovermode="closest",
    showlegend=False,
    title=dict(
        text="<b>Pengeluaran pelanggan berdasarkan wilayah</b>",
        font=dict(size=12, family="Arial", color="white"),
        x=0.01,
        y=0.95
    ),
    mapbox=dict(
        accesstoken=token,
        bearing=0,
        center=go.layout.mapbox.Center(lat=lat_center, lon=lon_center),
        pitch=0,
        zoom=3,
        style="dark"
    )
)

fig.show()


#### Pengeluaran pelanggan berdasarkan negara bagian

In [19]:
brazil_geojson_url = "https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/brazil-states.geojson"

with urlopen(brazil_geojson_url) as response:
    brazil_states = json.load(response)

data_customers_states = data_customers.groupby (by = "geolocation_state")[["orders_count", "total_value"]].sum().reset_index()

state_id_map = {}
for feature in brazil_states["features"]:
    feature["id"] = feature["properties"]["sigla"]
    state_id_map[feature["id"]] =  feature["properties"]["name"]

state_id_map

{'AC': 'Acre',
 'AL': 'Alagoas',
 'AM': 'Amazonas',
 'AP': 'Amapá',
 'BA': 'Bahia',
 'CE': 'Ceará',
 'ES': 'Espírito Santo',
 'GO': 'Goiás',
 'MA': 'Maranhão',
 'MG': 'Minas Gerais',
 'MS': 'Mato Grosso do Sul',
 'MT': 'Mato Grosso',
 'PA': 'Pará',
 'PB': 'Paraíba',
 'PE': 'Pernambuco',
 'PI': 'Piauí',
 'PR': 'Paraná',
 'RJ': 'Rio de Janeiro',
 'RN': 'Rio Grande do Norte',
 'RO': 'Rondônia',
 'RR': 'Roraima',
 'RS': 'Rio Grande do Sul',
 'SC': 'Santa Catarina',
 'SE': 'Sergipe',
 'SP': 'São Paulo',
 'TO': 'Tocantins',
 'DF': 'Distrito Federal'}

In [20]:
# Memetakan kode/nama provinsi pada kolom "geolocation_state" ke ID yang sesuai menggunakan state_id_map
# Lalu mengonversi ID hasil pemetaan menjadi tipe string
data_customers_states["name"] = data_customers_states["geolocation_state"].apply(
    lambda x: state_id_map[x]  # memetakan setiap provinsi ke ID yang sesuai
).astype(str)  # mengonversi hasilnya menjadi string

display(data_customers_states.head())

,geolocation_state,orders_count,total_value,name
0,AC,80,19575.33,Acre
1,AL,397,94172.49,Alagoas
2,AM,145,27585.47,Amazonas
3,AP,67,16141.81,Amapá
4,BA,3242,587373.55,Bahia


In [21]:
def world_map_transaction(df=data_customers_states, feature="name"):
    df_location = df.copy()

    # Menghitung perkiraan koordinat latitude/longitude untuk setiap state unik dalam data
    unique_state = df_location[feature].unique()
    state_coords = {}

    geolocator = Nominatim(user_agent="location_mapper")
    geocode = RateLimiter(
        geolocator.geocode,
        min_delay_seconds=1,
        max_retries=3,
        error_wait_seconds=3,
        swallow_exceptions=False
    )

    print(f"🔍 Mengambil koordinat untuk {len(unique_state)} state unik...")

    # Mengambil koordinat perkiraan untuk setiap state unik
    for state in unique_state:
        try:
            loc = geocode(f"{state}, Brazil", timeout=15)
            if loc:
                state_coords[state] = (loc.latitude, loc.longitude)
            else:
                print(f"⚠️ Koordinat tidak ditemukan untuk {state}")
        except Exception as e:
            print(f"❌ Terjadi error saat mengambil koordinat untuk {state}: {e}")

    # Menambahkan koordinat ke DataFrame
    df_location["name"] = df_location["name"].astype(str)
    df_location["Coordinates"] = df_location["name"].map(state_coords)
    df_location = df_location[
        df_location["Coordinates"].apply(lambda x: isinstance(x, (list, tuple)) and len(x) == 2)
    ]

    # Menginisialisasi peta Folium pada titik tengah perkiraan
    if len(state_coords) > 0:
        initial_coords = list(state_coords.values())[0]
    else:
        print("⚠️ Tidak ada koordinat ditemukan — peta diinisialisasi pada (0,0)")
        initial_coords = [0, 0]

    mymap = folium.Map(location=initial_coords, zoom_start=5, tiles="CartoDB dark_matter")

    for _, row in df_location.iterrows():
        coords = row["Coordinates"]

        tooltip_html = f"""
        <div style='font-size:13px; line-height:1.5'>
            <b>State:</b> {row['name']}<br>
            <b>Sigla:</b> {row['geolocation_state']}<br>
            <b>Jumlah Order:</b> {row['orders_count']}<br>
            <b>GMV:</b> ${row['total_value']:,.2f}
        </div>
        """

        # Menambahkan marker untuk setiap state pada peta
        folium.Marker(
            location=coords,
            tooltip=folium.Tooltip(tooltip_html, sticky=True),
            icon=folium.Icon(color="lightgray", icon="info-sign")
        ).add_to(mymap)

    map_path = f"./saved-map-brazil-olist.html"
    mymap.save(map_path)
    print(f"✅ Peta berhasil disimpan: {map_path}")

    return mymap

world_map_transaction()

🔍 Mengambil koordinat untuk 27 state unik...
✅ Peta berhasil disimpan: ./saved-map-brazil-olist.html


#### Pelanggan Kembali dan Pelanggan Baru

In [22]:
# Menggabungkan tabel orders dengan order items untuk mendapatkan detail level produk
# Hanya menyimpan order dengan status "delivered"
# Lalu menggabungkan dengan customers untuk mendapatkan informasi pelanggan
# Membuat kolom baru "year_month" dari timestamp pembelian order
# Memilih hanya kolom yang relevan untuk analisis deret waktu (time series)
data_timeseries = (
    df_orders
    .merge(df_order_items_, how="left", on="order_id")  # gabungkan orders dengan order items
    .query('order_status == "delivered"')  # filter hanya order yang sudah delivered
    .merge(df_customers, how="left", on="customer_id")  # gabungkan dengan customers
    .assign(year_month = lambda x: x["order_purchase_timestamp"].dt.to_period("M"))  # ekstrak periode tahun-bulan
    [["order_id", "customer_unique_id", "order_purchase_timestamp", "year_month", "product_counts", "total_price", "total_value"]]  # pilih kolom penting
)

# Membuat indeks urutan pembelian untuk setiap customer berdasarkan timestamp pembelian
# Mengurutkan dan memberi peringkat order secara kronologis untuk tiap customer
data_timeseries["order_index"] = data_timeseries.groupby("customer_unique_id")["order_purchase_timestamp"]\
                                               .rank(method="first", ascending=True)

display(data_timeseries.head())

,order_id,customer_unique_id,order_purchase_timestamp,year_month,product_counts,total_price,total_value,order_index
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,2017-10-02 10:56:33,2017-10,1.0,29.99,38.71,2.0
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,2018-07-24 20:41:37,2018-07,1.0,118.70,141.46,1.0
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,2018-08-08 08:38:49,2018-08,1.0,159.90,179.12,1.0
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,2017-11-18 19:28:06,2017-11,1.0,45.00,72.20,1.0
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,2018-02-13 21:18:39,2018-02,1.0,19.90,28.62,1.0


In [23]:
# Membuat ringkasan bulanan untuk pelanggan baru vs pelanggan kembali
data_customer_timeseries = (
    # Langkah 1: Mengakumulasikan pelanggan baru (order pertama dari setiap customer)
    data_timeseries.query("order_index == 1")  # pilih hanya order pertama
    .groupby("year_month")
    .agg(
        new_customers_count=("customer_unique_id", "nunique"),  # jumlah pelanggan baru
        new_customers_value=("total_value", "sum"),  # total nilai dari pelanggan baru
    )
    # Langkah 2: Menggabungkan dengan pelanggan kembali (order setelah order pertama)
    .merge(
        right=(
            data_timeseries.query("order_index > 1")  # pilih order berulang
            .groupby("year_month")
            .agg(
                return_customers_count=("customer_unique_id", "nunique"),  # jumlah pelanggan kembali
                return_customers_value=("total_value", "sum"),  # total nilai dari pelanggan kembali
            )
        ),
        how="left",  # pertahankan semua bulan dari pelanggan baru
        on="year_month"
    )
    .fillna(0)  # isi bulan tanpa pelanggan kembali dengan 0
    .reset_index()  # reset index untuk DataFrame yang rapi
)

display(data_customer_timeseries.head())

,year_month,new_customers_count,new_customers_value,return_customers_count,return_customers_value
0,2016-09,1,143.46,0.0,0.00
1,2016-10,262,44687.95,12.0,1802.71
2,2016-12,1,19.62,0.0,0.00
3,2017-01,717,121229.75,46.0,6252.62
4,2017-02,1628,262457.36,63.0,8781.96


In [24]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x = data_customer_timeseries["year_month"].astype(str),
        y = data_customer_timeseries["new_customers_count"],
        name = "Jumlah pelanggan baru",
        marker_color = "mediumaquamarine"
    )
)

fig.add_trace(
    go.Bar(
        x = data_customer_timeseries["year_month"].astype(str),
        y = data_customer_timeseries["return_customers_count"],
        name = "Jumlah pelanggan kembali",
        marker_color = "powderblue"
    )
)

fig.add_trace(
    go.Scatter(
        x = data_customer_timeseries["year_month"].astype(str),
        y = data_customer_timeseries["new_customers_value"],
        name = "Pendapatan dari pelanggan baru",
        yaxis = "y2",
        marker_color = "indianred",
        mode="lines+markers"
    )
)

fig.add_trace(
    go.Scatter(
        x = data_customer_timeseries["year_month"].astype(str),
        y = data_customer_timeseries["return_customers_value"],
        name = "Pendapatan dari pelanggan kembali",
        yaxis = "y2",
        marker_color = "sandybrown",
        mode="lines+markers"
    )
)

fig.update_layout(
    title = dict(text="<b>Pelanggan Baru dan Pelanggan Kembali Berdasarkan Bulan<b>",
                 font=dict(size=12, family="Arial", color="black")),
    plot_bgcolor = "white",
    barmode = "stack",
    yaxis = dict(side = "left", showgrid = False, zeroline = True, showline = False, showticklabels = False),
    yaxis2 = dict(side = "right", overlaying = "y", showgrid = False, zeroline = False, showline = False, showticklabels = False),
    xaxis = dict(showline = True, linecolor = "rgb(204, 204, 204)", linewidth = 2),
    legend = dict(orientation="h"),
    hovermode = "x unified",
    annotations=[dict(text="Created By AL.", xref="paper", yref="paper", x=1.05, y=-0.25,
    showarrow=False, font=dict(size=10, color="gray", family="Arial"))]
)
fig.show()

**Insight:**

- Secara umum, mayoritas pelanggan situs *e-commerce* ini adalah pelanggan baru dengan pembelian sekali seumur hidup. Sejak tahun 2017, situs ini telah berhasil menarik pelanggan baru dan pengeluaran dari kelompok ini menyumbang porsi yang besar.

- Di sisi lain, jumlah pelanggan yang kembali stabil selama periode tersebut, dan pengeluaran dari kelompok ini juga dapat diabaikan.

#### Nilai Seumur Hidup Pelanggan

In [25]:
# Mengakumulasikan metrik tingkat pelanggan dari data timeseries
data_orders = (
    data_timeseries
    .groupby("customer_unique_id")  # mengelompokkan data berdasarkan setiap customer unik
    .agg(
        last_purchase_date=("order_purchase_timestamp", "max"),  # tanggal pembelian terbaru
        order_count=("order_id", "nunique"),  # total jumlah pesanan
        quantity=("product_counts", "sum"),  # total produk yang dibeli
        total_price=("total_price", "sum"),  # total harga dari seluruh pesanan
        total_value=("total_value", "sum")   # total nilai dari seluruh pesanan
    )
    .sort_values(by="order_count", ascending=False)  # mengurutkan customer berdasarkan jumlah pesanan, dari terbesar
    .reset_index()  # reset index untuk mengubah customer_unique_id menjadi kolom
)

data_orders.head()

,customer_unique_id,last_purchase_date,order_count,quantity,total_price,total_value
0,8d50f5eadf50201ccdcedfb9e2ac8455,2018-08-20 19:14:26,15,15.0,714.63,879.27
1,3e43e6105506432c953e165fb2acf44c,2018-02-27 18:36:39,9,14.0,1000.85,1172.67
2,6469f99c1f9dfae7733b25662e7f1782,2018-06-28 00:43:34,7,9.0,664.20,758.83
3,ca77025e7201e3b30c44b472ff346268,2018-06-01 11:38:29,7,12.0,806.61,1122.72
4,1b6c7548a2a1f9037c1fd3ddfed95f33,2018-02-14 13:22:12,7,9.0,809.21,959.01


In [26]:
# Menghitung Average Order Value (AOV) untuk setiap pelanggan
# AOV = total nilai seluruh pesanan / jumlah pesanan
data_orders["AOV"] = data_orders["total_value"] / data_orders["order_count"]

# Menghitung frekuensi pembelian secara keseluruhan di seluruh pelanggan
# purchase_freq = total jumlah pesanan / total jumlah pelanggan unik
purchase_freq = data_orders["order_count"].sum() / len(data_orders)

# Menghitung repeat rate: proporsi pelanggan yang melakukan lebih dari satu pesanan
repeat_rate = data_orders[data_orders["order_count"] > 1].shape[0] / data_orders.shape[0]

# Menghitung churn rate: proporsi pelanggan yang tidak melakukan pembelian ulang
churn_rate = 1 - repeat_rate

# Mengestimasi profit margin (diasumsikan 10% dari total price)
data_orders["profit_margin"] = data_orders["total_price"] * 0.1

# Menghitung Customer Lifetime Value (CLV)
# CLV = (AOV × purchase frequency) / churn rate × 100
# Faktor (×100) digunakan agar skala CLV lebih mudah diinterpretasikan
data_orders["CLV"] = (data_orders["AOV"] * purchase_freq) / churn_rate * 100

q33 = data_orders["CLV"].quantile(0.33)
q66 = data_orders["CLV"].quantile(0.66)

# Fungsi untuk mengklasifikasikan pelanggan berdasarkan CLV
def clv_segment(x):
    if x <= q33:
        return "Low Value"
    elif x <= q66:
        return "Medium Value"
    else:
        return "High Value"

# Menambahkan kolom Segment
data_orders["CLV_Segment"] = data_orders["CLV"].apply(clv_segment)

# Menampilkan pelanggan dengan nilai CLV tertinggi (pelanggan VIP)
print("Top 10 High-Value Customers:")
display(data_orders.sort_values(by="CLV", ascending=False).reset_index(drop=True).head(10))

Top 10 High-Value Customers:


,customer_unique_id,last_purchase_date,order_count,quantity,total_price,total_value,AOV,profit_margin,CLV,CLV_Segment
0,0a0a92112bd4c708ca5fde585afaa872,2017-09-29 15:24:52,1,8.0,13440.00,13664.08,13664.08,1344.000,1.455750e+06,High Value
1,763c8b1c9c68a0229c42c9fc6f662b93,2018-07-15 14:49:44,1,4.0,7160.00,7274.88,7274.88,716.000,7.750542e+05,High Value
2,dc4802a71eae9be1dd28f5d788ceb526,2017-02-12 20:37:36,1,1.0,6735.00,6929.31,6929.31,673.500,7.382378e+05,High Value
3,459bef486812aa25204be022145caa62,2018-07-25 18:10:17,1,1.0,6729.00,6922.21,6922.21,672.900,7.374813e+05,High Value
4,ff4159b92c40ebe40454e3e6a7c35ed6,2017-05-24 18:14:34,1,1.0,6499.00,6726.66,6726.66,649.900,7.166478e+05,High Value
5,4007669dec559734d6f53e029e360987,2017-11-24 11:03:35,1,6.0,5934.60,6081.54,6081.54,593.460,6.479177e+05,High Value
6,eebb5dda148d3893cdaf5b5ca3040ccb,2017-04-18 18:50:13,1,1.0,4690.00,4764.34,4764.34,469.000,5.075853e+05,High Value
7,48e1ac109decbb87765a3eade6854098,2018-06-22 12:23:19,1,1.0,4590.00,4681.78,4681.78,459.000,4.987895e+05,High Value
8,edde2314c6c30e864a128ac95d6b2112,2018-08-03 21:10:16,1,1.0,4399.87,4513.32,4513.32,439.987,4.808420e+05,High Value
9,a229eba70ec1c2abef51f04987deb7a5,2018-05-31 22:57:07,1,2.0,4400.00,4445.50,4445.50,440.000,4.736166e+05,High Value


In [27]:
# Menghitung jumlah pelanggan di setiap segmen CLV (Low, Medium, High)
clv_group_dist = (data_orders["CLV_Segment"].value_counts().reset_index())

# Membuat layout visualisasi dengan 2 subplot: Pie chart + Bar chart
fig = make_subplots(
    rows=1,
    cols=2,
    specs=[[{'type':'domain'}, {'type':'xy'}]]
)

# --- PIE CHART ---
# Menambahkan pie chart untuk menunjukkan proporsi pelanggan di tiap segmen CLV
fig.add_trace(
    go.Pie(
        labels= clv_group_dist["CLV_Segment"],   # nama segmen
        values= clv_group_dist["count"],         # jumlah pelanggan per segmen
        textinfo="label+percent",                # tampilkan label dan persen
        hovertemplate="Group: %{label}<br>Number of Customers: %{value}",
        marker=dict(
            colors=["#E74C3C", "#F1C40F", "#27AE60"],   # warna segmen
            line=dict(color="white", width=2)           # garis putih pembatas
        ),
        name="CLV Segment Distribution",
        hole=0.8                                        # membuat pie menjadi donut chart
    ),
    row=1, col=1
)

# --- BAR CHART ---
# Menambahkan bar chart untuk menampilkan jumlah pelanggan per segmen CLV secara absolut
fig.add_trace(
    go.Bar(
        x = clv_group_dist["CLV_Segment"].astype(str),  # sumbu X = kategori segmen
        y = clv_group_dist["count"],                    # sumbu Y = jumlah pelanggan
        text=clv_group_dist["count"],                   # tampilkan angka di atas bar
        textposition="auto",                            # posisi teks otomatis
        marker=dict(
            color=["#E74C3C", "#F1C40F", "#27AE60"],    # warna konsisten dengan pie chart
            line=dict(width=1, color="white")           # garis tepi bar
        ),
        name="Count Plot of CLV Segment Distribution"
    ),
    row=1, col=2
)

# --- LAYOUT ---
# Mengatur layout tampilan grafik
fig.update_layout(
    title=dict(
        text="<b>Customer Lifetime Value (CLV) Segmentation Overview<b>",
        font=dict(size=12, family="Arial", color="black")
    ),
    plot_bgcolor="white",  # latar belakang putih
    xaxis=dict(
        showline=True, linecolor="rgb(204, 204, 204)", linewidth=2
    ),
    yaxis=dict(
        side="left", showgrid=False, zeroline=True, showline=False, showticklabels=False
    ),
    showlegend=False,  # legend tidak diperlukan karena label sudah jelas
    annotations=[
        dict(
            text="Created By AL.",
            xref="paper", yref="paper",
            x=1, y=-.2,
            showarrow=False,
            font=dict(size=10, color="gray")
        )
    ]
)

# Menampilkan grafik
fig.show()

**Insight:**

- **Pelanggan Bernilai Tinggi** mencakup **34%** dari basis pelanggan, dengan jumlah terbesar (**31.736 pelanggan**) — target utama untuk retensi dan upselling.

- Segmen **Nilai Sedang** dan **Nilai Rendah** hampir sama besarnya (~33%), masing-masing memiliki lebih dari **30.800 pelanggan** — ideal untuk strategi pembinaan dan pertumbuhan.

- Distribusi yang seimbang di seluruh segmen menunjukkan peluang untuk keterlibatan yang dipersonalisasi dan pendekatan pemasaran berjenjang.

### Analisis Produk

In [28]:
# Menggabungkan data level produk dan merangkumnya berdasarkan kategori
data_products = (
    # Langkah 1: Menghitung total jumlah produk dan total harga berdasarkan product_id
    df_order_items_.groupby("product_id")[["product_counts", "total_price"]].sum()
    # Langkah 2: Menggabungkan dengan detail produk (misalnya kategori) dari df_products
    .merge(df_products, how="left", on="product_id")
    # Langkah 3: Mengelompokkan ulang berdasarkan kategori produk
    .groupby("product_category_name")[["product_counts", "total_price"]].sum()
    # Langkah 4: Mengurutkan kategori berdasarkan total jumlah produk secara menurun
    .sort_values(by="product_counts", ascending=False)
    .reset_index()  # Reset index agar DataFrame lebih rapi
)

# Membersihkan nama kategori produk agar lebih mudah dibaca
# Mengganti tanda underscore dengan spasi dan membuat huruf awal setiap kata menjadi kapital
data_products["product_category"] = data_products["product_category_name"].apply(
    lambda x: str(x).replace("_", " ").title()
)

data_products.head()

,product_category_name,product_counts,total_price,product_category
0,cama_mesa_banho,11115,1036988.68,Cama Mesa Banho
1,beleza_saude,9670,1258681.34,Beleza Saude
2,esporte_lazer,8641,988048.97,Esporte Lazer
3,moveis_decoracao,8334,729762.49,Moveis Decoracao
4,informatica_acessorios,7827,911954.32,Informatica Acessorios


#### Produk Teratas Terjual Berdasarkan Kategori

In [75]:
fig = go.Figure()

# Menambahkan bar chart untuk menampilkan 15 kategori produk terlaris berdasarkan jumlah produk terjual
fig.add_trace(
    go.Bar(
        x=data_products["product_category"].head(15),   # 15 kategori produk teratas
        y=data_products["product_counts"].head(15),     # Jumlah produk terjual
        name="Jumlah produk yang terjual",                 # Label legend
        marker_color="rgb(64,224,208)"                  # Warna batang
    )
)

# Menambahkan line chart untuk menampilkan GMV dari 15 kategori produk terlaris
fig.add_trace(
    go.Scatter(
        x=data_products["product_category"].head(15),   # 15 kategori produk teratas
        y=data_products["total_price"].head(15),        # Total harga (GMV)
        name="GMV",                                     # Label legend
        yaxis="y2",                                     # Menggunakan sumbu Y kedua
        marker_color="rgb(255,160,122)",                # Warna garis/titik
        mode="lines+markers"                            # Garis dan marker
    )
)

# Mengatur layout, tampilan sumbu, judul, dan keterangan grafik
fig.update_layout(
    title=dict(
        text="<b>15 Kategori Produk Terlaris<b>",   # Judul grafik
        font=dict(size=12, family="Arial", color="black")  # Format teks judul
    ),
    plot_bgcolor="white",                                   # Warna latar belakang grafik
    yaxis=dict(                                             # Pengaturan sumbu Y (kiri)
        side="left",
        showgrid=False,
        zeroline=True,
        showline=False,
        showticklabels=False
    ),
    yaxis2=dict(                                            # Pengaturan sumbu Y kedua (kanan)
        side="right",
        overlaying="y",
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False
    ),
    xaxis=dict(                                             # Pengaturan sumbu X
        showline=True,
        linecolor="rgb(204, 204, 204)",
        linewidth=2
    ),
    legend=dict(                                            # Posisi legenda
        orientation="h",
        x=0.8,
        y=1.1
    ),
    annotations=[                                           # Catatan pembuat grafik
        dict(
            text="Created By AL.",
            xref="paper",
            yref="paper",
            x=1,
            y=-.3,
            showarrow=False,
            font=dict(size=10, color="gray", family="Arial")
        )
    ],
    hovermode="x unified"                                   # Mode hover agar rapi
)

# Menampilkan grafik
fig.show()

**Insight:**

* **Peralatan Perawatan Taman** memimpin dalam hal volume penjualan dan GMV → kategori produk inti.

* **Perlengkapan Kecantikan**, **Dekorasi Rumah** dan **Aksesori Kendaraan** juga memiliki pendapatan yang tinggi → permintaan konsumen yang kuat.

* Beberapa kategori seperti **Aksesori Ponsel** dan **Aksesori PC** memiliki volume penjualan yang tinggi tetapi GMV rendah → nilai pesanan rata-rata yang rendah.

* Sebaliknya, **Aksesori Furnitur** dan **Elektronik Kendaraan** memiliki GMV yang tinggi relatif terhadap volume penjualan → produk bernilai tinggi.

* Perbedaan antara volume dan GMV mencerminkan perilaku konsumen yang bervariasi di berbagai kategori.

#### Segmentasi Produk

In [30]:
# Menggabungkan data penjualan produk untuk pesanan yang telah dikirim (delivered)
data_products_seg = (
    # Langkah 1: Menggabungkan order items dengan orders, hanya menyertakan pesanan yang berstatus delivered
    df_order_items.merge(
        df_orders[["order_id", "order_status"]].query('order_status == "delivered"'),
        how="inner",
        on="order_id"
    )
    # Langkah 2: Mengelompokkan berdasarkan product_id untuk menghitung metrik penjualan
    .groupby("product_id")
    .agg(
        sales_volume=("product_id", "count"),  # jumlah item yang terjual
        sales_values=("price", "sum")          # total nilai penjualan
    )
    # Langkah 3: Mengurutkan produk berdasarkan total nilai penjualan secara menurun
    .sort_values(by="sales_values", ascending=False)
    .reset_index()  # reset index untuk DataFrame yang rapi
)

display(data_products_seg.head())

,product_id,sales_volume,sales_values
0,bb50f2e236e5eea0100680137654686c,194,63560.00
1,6cdd53843498f92890544667809f1595,153,53652.30
2,d6160fb7873f184099d9bc95e30376af,33,45949.35
3,d1c427060a0f73f6b889a5c7c61f2ac4,332,45620.56
4,99a4788cb24856965c36a24e339b6058,477,42049.66


In [31]:
# Membuat dataset riwayat penjualan untuk pesanan yang telah dikirim (delivered)
data_sales_his = (
    # Langkah 1: Memfilter hanya pesanan delivered dan memilih kolom yang relevan
    df_orders.query('order_status == "delivered"')[["order_id", "order_purchase_timestamp"]]
    # Langkah 2: Menggabungkan dengan order items untuk mendapatkan detail tingkat produk
    .merge(df_order_items[["order_id", "product_id"]], how="left", on="order_id")
)

# Mengambil hanya bagian tanggal dari timestamp untuk keperluan agregasi
data_sales_his["order_purchase_date"] = data_sales_his["order_purchase_timestamp"].dt.date

# Mengagregasi penjualan produk secara harian
data_sales_his = (
    data_sales_his[["product_id", "order_purchase_date"]]
    .groupby(["order_purchase_date", "product_id"])
    .agg(
        product_sold=("product_id", "count")  # menghitung jumlah produk terjual per hari
    )
    .reset_index()  # reset index agar DataFrame lebih rapi
)

display(data_sales_his.head())

,order_purchase_date,product_id,product_sold
0,2016-09-15,5a6b04657a4c5ee34285d1e4619a96b4,3
1,2016-10-03,107177bf61755f05c604fe57e02467d6,1
2,2016-10-03,3ae08df6bcbfe23586dd431c40bddbb7,1
3,2016-10-03,a5c3ddb1a400f50d1cf7138727aec136,1
4,2016-10-03,b72b39418216e944bb34e35f4d3ea8c7,1


In [32]:
# Memberikan peringkat pada setiap tanggal pembelian secara berurutan untuk membuat indeks "day"
data_sales_his["day"] = data_sales_his["order_purchase_date"].rank(method="dense", ascending=True).astype(int)

# Membuat pivot DataFrame sehingga satu baris per produk dan satu kolom per hari
# Nilai berisi jumlah produk yang terjual pada hari tersebut
data_sales_his = (
    data_sales_his
    .pivot(index="product_id", columns="day", values="product_sold")
    .fillna(0)  # mengisi nilai penjualan yang hilang dengan 0
    .astype(int)
    .reset_index()
)

# Mengganti nama kolom hari menjadi "day_1", "day_2", ...
data_sales_his.columns = [data_sales_his.columns[0]] + ["day_" + str(col) for col in data_sales_his.columns[1:]]

# Menyimpan kolom hari untuk perhitungan selanjutnya
sales_col = data_sales_his.columns[1:]

# Menggabungkan ringkasan penjualan tingkat produk (misalnya volume dan nilai penjualan)
data_sales_his = data_sales_his.merge(data_products_seg, how="left", on="product_id")

# Menghitung rata-rata penjualan harian untuk setiap produk
data_sales_his["mean"] = data_sales_his[sales_col].mean(axis=1)

# Menyimpan hanya produk yang memiliki penjualan
data_sales_his = data_sales_his.query("mean > 0")

# Menghitung standar deviasi penjualan harian untuk setiap produk
data_sales_his["std"] = data_sales_his[sales_col].std(axis=1)

# Menghitung koefisien variasi (std / mean) untuk setiap produk
# Ini mengukur variabilitas penjualan relatif terhadap rata-rata
data_sales_his["coef"] = data_sales_his["std"] / data_sales_his["mean"]

display(data_sales_his.head())

,product_id,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,day_32,day_33,day_34,day_35,day_36,day_37,day_38,day_39,day_40,day_41,day_42,day_43,day_44,day_45,day_46,day_47,day_48,day_49,day_50,day_51,day_52,day_53,day_54,day_55,day_56,day_57,day_58,day_59,day_60,day_61,day_62,day_63,day_64,day_65,day_66,day_67,day_68,day_69,day_70,day_71,day_72,day_73,day_74,day_75,day_76,day_77,day_78,day_79,day_80,day_81,day_82,day_83,day_84,day_85,day_86,day_87,day_88,day_89,day_90,day_91,day_92,day_93,day_94,day_95,day_96,day_97,day_98,day_99,day_100,day_101,day_102,day_103,day_104,day_105,day_106,day_107,day_108,day_109,day_110,day_111,day_112,day_113,day_114,day_115,day_116,day_117,day_118,day_119,day_120,day_121,day_122,day_123,day_124,day_125,day_126,day_127,day_128,day_129,day_130,day_131,day_132,day_133,day_134,day_135,day_136,day_137,day_138,day_139,day_140,day_141,day_142,day_143,day_144,day_145,day_146,day_147,day_148,day_149,day_150,day_151,day_152,day_153,day_154,day_155,day_156,day_157,day_158,day_159,day_160,day_161,day_162,day_163,day_164,day_165,day_166,day_167,day_168,day_169,day_170,day_171,day_172,day_173,day_174,day_175,day_176,day_177,day_178,day_179,day_180,day_181,day_182,day_183,day_184,day_185,day_186,day_187,day_188,day_189,day_190,day_191,day_192,day_193,day_194,day_195,day_196,day_197,day_198,day_199,day_200,day_201,day_202,day_203,day_204,day_205,day_206,day_207,day_208,day_209,day_210,day_211,day_212,day_213,day_214,day_215,day_216,day_217,day_218,day_219,day_220,day_221,day_222,day_223,day_224,day_225,day_226,day_227,day_228,day_229,day_230,day_231,day_232,day_233,day_234,day_235,day_236,day_237,day_238,day_239,day_240,day_241,day_242,day_243,day_244,day_245,day_246,day_247,day_248,day_249,...,day_368,day_369,day_370,day_371,day_372,day_373,day_374,day_375,day_376,day_377,day_378,day_379,day_380,day_381,day_382,day_383,day_384,day_385,day_386,day_387,day_388,day_389,day_390,day_391,day_392,day_393,day_394,day_395,day_396,day_397,day_398,day_399,day_400,day_401,day_402,day_403,day_404,day_405,day_406,day_407,day_408,day_409,day_410,day_411,day_412,day_413,day_414,day_415,day_416,day_417,day_418,day_419,day_420,day_421,day_422,day_423,day_424,day_425,day_426,day_427,day_428,day_429,day_430,day_431,day_432,day_433,day_434,day_435,day_436,day_437,day_438,day_439,day_440,day_441,day_442,day_443,day_444,day_445,day_446,day_447,day_448,day_449,day_450,day_451,day_452,day_453,day_454,day_455,day_456,day_457,day_458,day_459,day_460,day_461,day_462,day_463,day_464,day_465,day_466,day_467,day_468,day_469,day_470,day_471,day_472,day_473,day_474,day_475,day_476,day_477,day_478,day_479,day_480,day_481,day_482,day_483,day_484,day_485,day_486,day_487,day_488,day_489,day_490,day_491,day_492,day_493,day_494,day_495,day_496,day_497,day_498,day_499,day_500,day_501,day_502,day_503,day_504,day_505,day_506,day_507,day_508,day_509,day_510,day_511,day_512,day_513,day_514,day_515,day_516,day_517,day_518,day_519,day_520,day_521,day_522,day_523,day_524,day_525,day_526,day_527,day_528,day_529,day_530,day_531,day_532,day_533,day_534,day_535,day_536,day_537,day_538,day_539,day_540,day_541,day_542,day_543,day_544,day_545,day_546,day_547,day_548,day_549,day_550,day_551,day_552,day_553,day_554,day_555,day_556,day_557,day_558,day_559,day_560,day_561,day_562,day_563,day_564,day_565,day_566,day_567,day_568,day_569,day_570,day_571,day_572,day_573,day_574,day_575,day_576,day_577,day_578,day_579,day_580,day_581,day_582,day_583,day_584,day_585,day_586,day_587,day_588,day_589,day_590,day_591,day_592,day_593,day_594,day_595,day_596,day_597,day_598,day_599,day_600,day_601,day_602,day_603,day_604,day_605,day_606,day_607,day_608,day_609,day_610,day_611,day_612,sales_volume,sales_values,mean,std,coef
0,00066f42aeeb9f3007548bb9d3f33c38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [33]:
# Membuat ringkasan penjualan tingkat produk dengan menghapus kolom penjualan harian
data_product_summary = data_sales_his.copy().drop(columns=sales_col, axis=1)

# Menghitung total nilai penjualan dari seluruh produk
total_sales_values = data_product_summary["sales_values"].sum()

# Menghitung persentase kontribusi setiap produk terhadap total penjualan
data_product_summary["%sales"] = data_product_summary["sales_values"] / total_sales_values * 100

# Mengurutkan produk berdasarkan persentase kontribusi penjualan dari yang terbesar
data_product_summary.sort_values(by="%sales", ascending=False, inplace=True, ignore_index=True)

# Menghitung persentase penjualan kumulatif (berguna untuk analisis Pareto/ABC)
data_product_summary["%sales_cs"] = data_product_summary["%sales"].cumsum()

display(data_product_summary.head())

,product_id,sales_volume,sales_values,mean,std,coef,%sales,%sales_cs
0,bb50f2e236e5eea0100680137654686c,194,63560.00,0.316993,0.728767,2.298998,0.480732,0.480732
1,6cdd53843498f92890544667809f1595,153,53652.30,0.250000,0.614540,2.458161,0.405796,0.886528
2,d6160fb7873f184099d9bc95e30376af,33,45949.35,0.053922,0.331689,6.151316,0.347535,1.234063
3,d1c427060a0f73f6b889a5c7c61f2ac4,332,45620.56,0.542484,0.987562,1.820446,0.345048,1.579112
4,99a4788cb24856965c36a24e339b6058,477,42049.66,0.779412,1.211795,1.554755,0.318040,1.897152


In [34]:
# Jumlah total produk
n_products = len(data_product_summary)

# Menentukan jumlah produk dalam setiap kategori ABC
# 5% teratas = A, 15% berikutnya = B, sisanya = C
n_a, n_b = int(0.05 * n_products), int(0.2 * n_products)

# Memberikan SKU ID unik untuk setiap produk
data_product_summary["sku_id"] = pd.Series(range(1, n_products + 1)).astype(int)

# Menetapkan kategori ABC berdasarkan peringkat produk
# - "A": 5% produk teratas berdasarkan kontribusi penjualan
# - "B": 15% produk berikutnya
# - "C": produk sisanya
data_product_summary["abc"] = pd.Series(range(n_products)).apply(
    lambda x: "A" if x <= n_a - 1 else "B" if x <= n_b - 1 else "C"
)

display(data_product_summary.head())

,product_id,sales_volume,sales_values,mean,std,coef,%sales,%sales_cs,sku_id,abc
0,bb50f2e236e5eea0100680137654686c,194,63560.00,0.316993,0.728767,2.298998,0.480732,0.480732,1,A
1,6cdd53843498f92890544667809f1595,153,53652.30,0.250000,0.614540,2.458161,0.405796,0.886528,2,A
2,d6160fb7873f184099d9bc95e30376af,33,45949.35,0.053922,0.331689,6.151316,0.347535,1.234063,3,A
3,d1c427060a0f73f6b889a5c7c61f2ac4,332,45620.56,0.542484,0.987562,1.820446,0.345048,1.579112,4,A
4,99a4788cb24856965c36a24e339b6058,477,42049.66,0.779412,1.211795,1.554755,0.318040,1.897152,5,A


In [35]:
# Loop melalui setiap kelas produk ABC dan rangkum kontribusinya
for product_class in ["A", "B", "C"]:
    # Filter produk yang termasuk dalam kelas saat ini
    filter = data_product_summary[data_product_summary["abc"] == product_class]

    # Hitung persentase produk dalam kelas ini
    percentage = len(filter) / len(data_product_summary) * 100

    # Dapatkan SKU ID tertinggi dalam kelas ini (sebagai proxy jumlah SKU)
    sku_number = filter["sku_id"].max()

    # Hitung persentase total penjualan yang dikontribusikan oleh kelas ini
    sales_percent = filter["%sales"].sum()

    # Cetak ringkasan dari kelas tersebut
    print(f"Class {product_class} ({percentage:.0f}%) memiliki {sku_number} SKU dan {sales_percent:.2f}% dari total penjualan")

Class A (5%) memiliki 1610 SKU dan 47.82% dari total penjualan
Class B (15%) memiliki 6443 SKU dan 27.00% dari total penjualan
Class C (80%) memiliki 32216 SKU dan 25.18% dari total penjualan


In [36]:
# Salin data ringkasan produk ke variabel baru untuk plotting
data_product_plot = data_product_summary.copy()

# Pemetaan kelas produk ABC ke warna masing-masing
class_mapping = {"A": "mediumturquoise", "B": "indianred", "C": "limegreen"}

# Inisialisasi figure Plotly
fig = go.Figure()

# Tambahkan scatter plot untuk setiap kelas produk (A, B, C)
for prod_class in class_mapping.keys():
    fig.add_trace(
        go.Scatter(
            # Sumbu X: persentase kontribusi penjualan (%sales)
            x=data_product_plot[data_product_plot["abc"] == prod_class]["%sales"],
            # Sumbu Y: koefisien variasi permintaan (coef)
            y=data_product_plot[data_product_plot["abc"] == prod_class]["coef"],
            name="Produk Kelas " + str(prod_class),   # Nama kategori pada legend
            marker=dict(color=class_mapping[prod_class]),  # Warna berdasarkan kelas ABC
            mode="markers"   # Mode tampilan sebagai titik-titik
        )
    )

# Pengaturan tampilan layout grafik
fig.update_layout(
    # Judul grafik
    title=dict(
        text="<b>Distribusi Berdasarkan Variabilitas Permintaan<b>",
        font=dict(size=12, family="Arial", color="black")
    ),

    # Warna background plot
    plot_bgcolor="white",

    # Konfigurasi sumbu Y
    yaxis=dict(
        side="left",
        linecolor="rgb(204, 204, 204)",
        showline=True,
        linewidth=2,
        title="Variabilitas Permintaan",   # Judul sumbu Y
        range=[
            math.floor(min(data_product_summary["coef"])),
            math.ceil(max(data_product_summary["coef"])) + 1
        ]  # Rentang sumbu Y berdasarkan nilai coef
    ),

    # Konfigurasi sumbu X
    xaxis=dict(
        showline=True,
        linecolor="rgb(204, 204, 204)",
        linewidth=2,
        title="Persentase Omzet (%)",  # Judul sumbu X
        range=[
            math.floor(min(data_product_summary["%sales"])) - 0.01,
            (max(data_product_summary["%sales"])) + 0.01
        ]  # Rentang sumbu X berdasarkan nilai %sales
    ),

    # Penempatan legend secara horizontal
    legend=dict(orientation="h"),

    # Tambahkan anotasi pembuat grafik
    annotations=[
        dict(
            text="Created By AL.",
            xref="paper", yref="paper",
            x=1, y=-.2,
            showarrow=False,
            font=dict(size=10, color="gray", family="Arial")
        )
    ]
)

# Tampilkan grafik
fig.show()

In [37]:
# Membuat layout subplot dengan 1 baris dan 3 kolom untuk kelas A, B, dan C
fig = make_subplots(
    rows=1, cols=3,
    start_cell="top-left",
    subplot_titles=("Kelas A", "Kelas B", "Kelas C")  # Judul untuk masing-masing subplot
)

# Daftar kelas produk ABC
product_classes = ["A", "B", "C"]
col = 1  # Penanda kolom subplot yang sedang diproses

# Loop untuk membuat histogram per kelas produk
for product_class in product_classes:

    # Ambil nilai sales_values untuk kelas produk tertentu
    x = data_product_summary[data_product_summary["abc"] == product_class]["sales_values"]

    # Membuat histogram untuk kelas tersebut
    trace = go.Histogram(
        x=x,
        nbinsx=50,  # Jumlah bin histogram
        marker=dict(
            # Warna histogram berdasarkan kelas ABC
            color="mediumturquoise" if product_class == "A"
                  else "indianred" if product_class == "B"
                  else "limegreen"
        )
    )

    # Menambahkan trace histogram ke subplot yang sesuai
    fig.append_trace(trace, 1, col)

    # Mengatur teks judul sumbu X pada subplot tersebut
    fig.update_xaxes(
        title_text="Nilai Penjualan",
        title_font=dict(size=10, family="Arial", color="black"),
        row=1, col=col
    )

    # Mengatur teks judul sumbu Y pada subplot tersebut
    fig.update_yaxes(
        title_text="Jumlah Produk",
        title_font=dict(size=10, family="Arial", color="black"),
        row=1, col=col
    )

    # Pindah ke kolom subplot berikutnya
    col += 1

# Mengatur layout tampilan keseluruhan grafik
fig.update_layout(
    title=dict(
        text="<b>Distribusi Penjualan Berdasarkan Kelas Produk<b>",  # Judul utama grafik
        font=dict(size=12, family="Arial", color="black")
    ),
    plot_bgcolor="white",  # Warna background plot
    showlegend=False,      # Tidak menampilkan legend
    bargap=0.01            # Jarak antar bar histogram
)

# Menambahkan anotasi pembuat grafik
fig.add_annotation(
    text="Created By AL.",
    xref="paper", yref="paper",
    x=1, y=-0.2,
    showarrow=False,
    font=dict(size=10, color="gray", family="Arial")
)

# Menampilkan grafik
fig.show()

**Insight:**

* **Kelas A** → produk bernilai tinggi (hingga 60K) tetapi sebagian besar penjualannya rendah → miring ke kanan, beberapa penjual teratas mendorong pendapatan.
* **Kelas B** → penjualan sedang, merata (0-1.4K) → kategori stabil.
* **Kelas C** → produk bernilai rendah (maksimal ~400) tetapi banyak item → frekuensi tinggi, nilai pesanan rendah.

In [38]:
# Menghitung ambang volatilitas sebagai titik tengah dari rentang nilai standar deviasi
volatility_threshold = (max(data_product_summary["std"]) - min(data_product_summary["std"])) / 2

# Mendefinisikan fungsi untuk mengklasifikasikan produk berdasarkan persentase penjualan dan volatilitas
def classify(product):
    if product["%sales"] <= 0.25 and product["std"] <= volatility_threshold:
        return "Low Volume, Low Volatility"
    elif product["%sales"] > 0.25 and product["std"] <= volatility_threshold:
        return "High Volume, Low Volatility"
    elif product["%sales"] <= 0.25 and product["std"] > volatility_threshold:
        return "Low Volume, High Volatility"
    else:
        return "High Volume, High Volatility"

# Menerapkan fungsi klasifikasi untuk setiap produk
data_product_summary["type"] = data_product_summary.apply(lambda x: classify(x), axis=1)

display(data_product_summary.head())

,product_id,sales_volume,sales_values,mean,std,coef,%sales,%sales_cs,sku_id,abc,type
0,bb50f2e236e5eea0100680137654686c,194,63560.00,0.316993,0.728767,2.298998,0.480732,0.480732,1,A,"High Volume, Low Volatility"
1,6cdd53843498f92890544667809f1595,153,53652.30,0.250000,0.614540,2.458161,0.405796,0.886528,2,A,"High Volume, Low Volatility"
2,d6160fb7873f184099d9bc95e30376af,33,45949.35,0.053922,0.331689,6.151316,0.347535,1.234063,3,A,"High Volume, Low Volatility"
3,d1c427060a0f73f6b889a5c7c61f2ac4,332,45620.56,0.542484,0.987562,1.820446,0.345048,1.579112,4,A,"High Volume, High Volatility"
4,99a4788cb24856965c36a24e339b6058,477,42049.66,0.779412,1.211795,1.554755,0.318040,1.897152,5,A,"High Volume, High Volatility"


In [39]:
# Mapping jenis produk dengan informasi tambahan, warna, dan koordinat area plot
type_mapping = {
    "Low Volume, Low Volatility": {
        "info" : "Easy/ Low ROI",           # Kategori mudah / ROI rendah
        "color": "powderblue",              # Warna area
        "x": [0, 2, 2, 0],                  # Koordinat sumbu X area
        "y": [0, 0, 2, 2]                   # Koordinat sumbu Y area
    },
    "Low Volume, High Volatility": {
        "info" : "Difficult/ Low ROI",      # Kategori sulit / ROI rendah
        "color": "orangered",
        "x": [0, 0, 2, 2],
        "y": [2, 4, 4, 2],
    },
    "High Volume, Low Volatility": {
        "info" : "Moderate+/ High ROI",     # Volume tinggi / volatilitas rendah
        "color": "skyblue",
        "x": [2, 4, 4, 2],
        "y": [0, 0, 2, 2],
    },
    "High Volume, High Volatility": {
        "info" : "Critical+/ Moderate+/ High ROI",   # Kategori kritis / ROI tinggi
        "color": "yellowgreen",
        "x": [2, 4, 4, 2],
        "y": [2, 2, 4, 4],
    }
}

# Fungsi untuk membuat area plot berdasarkan koordinat dan tipe kategori produk
def plot (x, y, _type_):
    return go.Scatter (
        x = x,
        y = y,
        fill = "toself",                            # Mengisi area polygon
        fillcolor = type_mapping[_type_]["color"],  # Warna area
        hoveron = "fills",                           # Hover pada area
        hoverinfo = "text",                          # Info hover berupa teks
        line_color = "white",                        # Warna garis batas
        mode = "lines+text",
        name = _type_,                               # Label legenda
    )

# Total nilai penjualan semua produk (untuk menghitung persentase kontribusi)
total_values = data_product_summary["sales_values"].sum()

# Fungsi untuk menambahkan anotasi ringkasan informasi pada setiap area kategori
def plot_annotation (x, y, _type_):
    # Hitung total produk unik pada kategori ini
    total_products_sold = data_product_summary[data_product_summary["type"] == _type_]["product_id"].nunique()
    # Hitung total volume penjualan (jumlah item terjual)
    total_sales_volume = data_product_summary[data_product_summary["type"] == _type_]["sales_volume"].sum()
    # Hitung total nilai penjualan
    total_sales_values = data_product_summary[data_product_summary["type"] == _type_]["sales_values"].sum()
    # Hitung persentase kontribusi penjualan
    values_percent = total_sales_values / total_values * 100
    # Ambil informasi ringkas kategori
    info = type_mapping[_type_]["info"]

    # Format teks anotasi
    text = (
        f"<b>{_type_}</b><br>{info}"
        f"<br>Total Products Sold: {total_products_sold}"
        f"<br>Total Sales Volume: {total_sales_volume}"
        f"<br>Total Sales Value: {total_sales_values:.2f} ({values_percent:.2f}%)"
    )

    # Tambahkan anotasi ke grafik
    return fig.add_annotation(
        x = x, y = y, font = dict(color="black", size=12),
        text = text, align="left", xanchor="left", showarrow=False
    )

In [40]:
# Membuat objek figure kosong
fig = go.Figure()

# Loop untuk menggambar setiap area berdasarkan tipe produk
for _type_ in set(data_product_summary["type"]):
    x = type_mapping[_type_]["x"]      # Mengambil koordinat X dari mapping
    y = type_mapping[_type_]["y"]      # Mengambil koordinat Y dari mapping

    # Menambahkan area polygon ke grafik
    fig.add_trace(
        plot(x, y, _type_)
    )

    # Menambahkan anotasi informasi di dalam area kategori
    plot_annotation(x[0] + 0.05, y[0] + 0.7, _type_)

# Konfigurasi tampilan layout plot
fig.update_layout(
    title=dict(
        text="<b>Distribusi Penjualan Berdasarkan Kelas Produk<b>",  # Judul grafik
        font=dict(size=12, family="Arial", color="black")
    ),
    plot_bgcolor="white",                                  # Latar belakang putih
    xaxis=dict(
        showline=False, range=[0, 4], rangemode="nonnegative"  # Pengaturan sumbu X
    ),
    yaxis=dict(
        showline=False, range=[0, 4], rangemode="nonnegative", # Pengaturan sumbu Y
        tickmode="array", tickvals=[1, 2, 3, 4]                # Titik label pada sumbu Y
    ),
    showlegend=False                                         # Sembunyikan legenda
)

# Menambahkan anotasi pembuat grafik
fig.add_annotation(
    text="Created By AL.",
    xref="paper", yref="paper",
    x=1, y=-0.2,
    showarrow=False,
    font=dict(size=10, color="gray", family="Arial")
)

# Menampilkan plot
fig.show()

## Analisis Lanjutan

### Segmentasi Pelanggan Menggunakan Model RFM

In [41]:
# Membuat DataFrame baru untuk analisis RFM dengan kolom yang dipilih
df_rfm = (
    data_orders[["customer_unique_id", "last_purchase_date", "order_count", "total_price"]].copy()
    .sort_values(by="last_purchase_date", ascending=True)  # mengurutkan berdasarkan tanggal pembelian terakhir (ascending)
)

# Mengganti nama kolom ke terminologi standar RFM
df_rfm.columns = ["customer_unique_id", "last_purchase_date", "freq", "monetary"]

# Menentukan tanggal acuan untuk perhitungan recency
# Tambahan +1 hari untuk menghindari Recency = 0 bagi pelanggan dengan transaksi di tanggal terbaru
recency_date = df_rfm["last_purchase_date"].max() + pd.Timedelta(days=1)

# Menghitung recency (jumlah hari sejak pembelian terakhir)
df_rfm["recency"] = df_rfm["last_purchase_date"].apply(
    lambda x: (recency_date - x).days
).fillna(0).astype(int)

display(df_rfm.head())

,customer_unique_id,last_purchase_date,freq,monetary,recency
17247,830d5b7aaa3b6f1e9ad63703bec97d23,2016-09-15 12:16:38,1,134.97,714
48926,2f64e403852e6893ae37485d5fcacdaf,2016-10-03 16:56:50,1,21.90,695
74106,61db744d2f835035a5625b59350c6b63,2016-10-03 21:13:36,1,36.49,695
13067,8d3a54507421dbd2ce0a1d58046826e0,2016-10-03 22:06:03,1,119.90,695
18629,87776adb449c551e74c13fc34f036105,2016-10-03 22:31:31,1,29.99,695


In [42]:
# Skor Recency: recency lebih kecil = lebih baik (pembelian lebih baru)
def r_score(value, r_quartiles):
    if value >= r_quartiles[0.8]:
        return 1
    elif value >= r_quartiles[0.6]:
        return 2
    elif value >= r_quartiles[0.4]:
        return 3
    elif value >= r_quartiles[0.2]:
        return 4
    else:
        return 5

# Skor Frequency: frekuensi lebih tinggi = lebih baik
def f_score(value, f_quartiles):
    if value >= f_quartiles[0.8]:
        return 5
    elif value >= f_quartiles[0.6]:
        return 4
    elif value >= f_quartiles[0.4]:
        return 3
    elif value >= f_quartiles[0.2]:
        return 2
    else:
        return 1

# Skor Monetary: nilai monetari lebih tinggi = lebih baik
def m_score(value, m_quartiles):
    if value >= m_quartiles[0.8]:
        return 5
    elif value >= m_quartiles[0.6]:
        return 4
    elif value >= m_quartiles[0.4]:
        return 3
    elif value >= m_quartiles[0.2]:
        return 2
    else:
        return 1

# Menghitung kuartil untuk Recency, Frequency, Monetary
r_quartiles = df_rfm["recency"].quantile([0.2, 0.4, 0.6, 0.8])
f_quartiles = df_rfm["freq"].quantile([0.2, 0.4, 0.6, 0.8])
m_quartiles = df_rfm["monetary"].quantile([0.2, 0.4, 0.6, 0.8])

# Menerapkan fungsi perhitungan skor ke setiap pelanggan
df_rfm["R_Score"] = df_rfm["recency"].apply(lambda x: r_score(x, r_quartiles))
df_rfm["F_Score"] = df_rfm["freq"].apply(lambda x: f_score(x, f_quartiles))
df_rfm["M_Score"] = df_rfm["monetary"].apply(lambda x: m_score(x, m_quartiles))

# Menggabungkan skor R, F, M menjadi satu skor RFM dalam bentuk string
df_rfm["RFM_Score"] = (
    df_rfm["R_Score"].astype(str) +
    df_rfm["F_Score"].astype(str) +
    df_rfm["M_Score"].astype(str)
)

display(df_rfm.head())

,customer_unique_id,last_purchase_date,freq,monetary,recency,R_Score,F_Score,M_Score,RFM_Score
17247,830d5b7aaa3b6f1e9ad63703bec97d23,2016-09-15 12:16:38,1,134.97,714,1,5,4,154
48926,2f64e403852e6893ae37485d5fcacdaf,2016-10-03 16:56:50,1,21.90,695,1,5,1,151
74106,61db744d2f835035a5625b59350c6b63,2016-10-03 21:13:36,1,36.49,695,1,5,1,151
13067,8d3a54507421dbd2ce0a1d58046826e0,2016-10-03 22:06:03,1,119.90,695,1,5,4,154
18629,87776adb449c551e74c13fc34f036105,2016-10-03 22:31:31,1,29.99,695,1,5,1,151


In [43]:
# mendefinisikan fungsi untuk menetapkan segmen berdasarkan skor RFM
def rfm_segment(score):
    if score in ["555", "554", "545", "544", "545", "455", "445"]:
        return "Champions"
    elif score in ["543", "444", "435", "355", "354", "345", "344", "335"]:
        return "Loyal"
    elif score in ["553", "551", "552", "541", "542", "533", "532", "531", "452", "451", "442", "441",
                   "431", "453", "433", "432", "423", "353", "352", "351", "342", "341", "333", "323"]:
        return "Potential Loyalist"
    elif score in ["525", "524", "523", "522", "521", "515", "514", "513", "425", "424", "413", "414",
                   "415", "315", "314", "313"]:
        return "Promising"
    elif score in ["512", "511", "422", "421", "412", "411", "311"]:
        return "New Customers"
    elif score in ["535", "534", "443", "434", "343", "334", "325", "324"]:
        return "Need Attention"
    elif score in ["331", "321", "312", "221", "213", "231", "241", "251"]:
        return "About To Sleep"
    elif score in ["255", "254", "245", "244", "243", "252", "243", "242", "235", "234", "225", "224",
                   "153", "152", "145", "143", "142", "135", "134", "133", "125", "124"]:
        return "At Risk"
    elif score in ["155", "154", "144", "214", "215", "115", "114", "113"]:
        return "Cannot Lose Them"
    elif score in ["332", "322", "233", "232", "223", "222", "132", "123", "122", "212", "211"]:
        return "Hibernating Customers"
    elif score in ["111", "112", "121", "131", "141", "151"]:
        return "Lost Customers"
    else:
        return "Other"

df_rfm["Segment"] = df_rfm["RFM_Score"].apply(rfm_segment)
display(df_rfm.head())

,customer_unique_id,last_purchase_date,freq,monetary,recency,R_Score,F_Score,M_Score,RFM_Score,Segment
17247,830d5b7aaa3b6f1e9ad63703bec97d23,2016-09-15 12:16:38,1,134.97,714,1,5,4,154,Cannot Lose Them
48926,2f64e403852e6893ae37485d5fcacdaf,2016-10-03 16:56:50,1,21.90,695,1,5,1,151,Lost Customers
74106,61db744d2f835035a5625b59350c6b63,2016-10-03 21:13:36,1,36.49,695,1,5,1,151,Lost Customers
13067,8d3a54507421dbd2ce0a1d58046826e0,2016-10-03 22:06:03,1,119.90,695,1,5,4,154,Cannot Lose Them
18629,87776adb449c551e74c13fc34f036105,2016-10-03 22:31:31,1,29.99,695,1,5,1,151,Lost Customers


In [44]:
# Menggabungkan data RFM berdasarkan Segmen pelanggan
data_rfm_summary = (
    df_rfm
    .groupby("Segment")  # kelompokkan berdasarkan segmen pelanggan
    .agg(
        customer_count=("Segment", "count"),  # jumlah pelanggan pada setiap segmen
        total_monetary=("monetary", "sum")    # total nilai monetary pada setiap segmen
    )
    .reset_index()  # reset index agar DataFrame lebih rapi
)

# Hitung total nilai monetary dari seluruh segmen
total_monetary_all = data_rfm_summary["total_monetary"].sum()

# Hitung persentase kontribusi setiap segmen terhadap total monetary keseluruhan
data_rfm_summary["total_monetary_percent"] = round(
    (data_rfm_summary["total_monetary"] / total_monetary_all) * 100, 2
)

# Skala nilai total monetary ke rentang [0,1] menggunakan fungsi scaling yang telah didefinisikan sebelumnya
data_rfm_summary["total_monetary_scaling"] = data_rfm_summary["total_monetary"].apply(
    lambda x: scaling(x, data_rfm_summary, "total_monetary")
)

display(data_rfm_summary.head())

,Segment,customer_count,total_monetary,total_monetary_percent,total_monetary_scaling
0,About To Sleep,3509,90078.94,0.68,0.000000
1,At Risk,19143,2830407.45,21.41,0.778986
2,Cannot Lose Them,7290,2001949.19,15.14,0.543482
3,Champions,11432,3607892.80,27.29,1.000000
4,Lost Customers,3873,97266.38,0.74,0.002043


In [45]:
# Buat dictionary untuk memetakan setiap segmen pelanggan ke warna,
# berdasarkan nilai total monetary yang telah diskalakan
color = {}

# Iterasi setiap segmen dan nilai total monetary yang telah diskalakan
for ele in data_rfm_summary[["Segment", "total_monetary_scaling"]].to_dict("records"):
    if ele["total_monetary_scaling"] <= 0.1:
        color[ele["Segment"]] = "#ffcab3"  # kontribusi sangat rendah
    elif ele["total_monetary_scaling"] <= 0.3:
        color[ele["Segment"]] = "#ffb999"  # kontribusi rendah
    elif ele["total_monetary_scaling"] <= 0.5:
        color[ele["Segment"]] = "#ff9e80"  # kontribusi menengah
    elif ele["total_monetary_scaling"] <= 0.8:
        color[ele["Segment"]] = "#ff9566"  # kontribusi tinggi
    else:
        color[ele["Segment"]] = "#ff6119"  # kontribusi sangat tinggi

In [46]:
# Fungsi untuk membuat anotasi (keterangan teks) pada plot
# x, y : posisi anotasi
# name : nama segmen pelanggan
def annotation (x, y, name):

    # Ambil total monetary untuk segmen tertentu dan bulatkan 2 digit
    monetary = round(float(data_rfm_summary[data_rfm_summary["Segment"] == name]["total_monetary"]), 2)

    # Ambil persentase kontribusi monetary untuk segmen tersebut
    monetary_percent = round(float(data_rfm_summary[data_rfm_summary["Segment"] == name]["total_monetary_percent"]), 2)

    # Ambil jumlah pelanggan di segmen tersebut
    customers_count = int(data_rfm_summary[data_rfm_summary["Segment"] == name]["customer_count"])

    # Format teks anotasi
    text = f"<b>{name}</b><br>Total Customers: {customers_count}<br>Total Monetary: {monetary} ({monetary_percent}%)"

    # Tambahkan anotasi ke dalam figure dengan posisi dan gaya tertentu
    return fig.add_annotation (
        x = x, y = y, xref = "x domain", yref= "y domain", font = dict(color = "black", size = 11),
        text = text, align= "left", xanchor = "left", showarrow = False)


# Fungsi untuk membuat bentuk (polygon) representasi segmen pada radar chart
# x, y : koordinat titik-titik polygon
# name : nama segmen pelanggan
def property (x, y, name):
    return go.Scatter (
        x = x,                     # daftar nilai x untuk polygon
        y = y,                     # daftar nilai y untuk polygon
        fill = "toself",           # mengisi area polygon
        fillcolor = color[name],   # warna diambil dari color dictionary berdasarkan segmen
        hoveron = "fills",         # hover muncul saat mouse berada di area isi
        hoverinfo = "text",        # hanya teks yang muncul saat hover
        line_color = "white",      # warna garis tepi polygon
        mode = "lines+text",       # menampilkan garis + teks
        name = name                # nama segmen untuk legend
    )

In [76]:
# Membuat figure kosong untuk menampung seluruh shape segmen RFM
fig = go.Figure()

# Tambahkan polygon untuk segmen "Lost Customers"
fig.add_trace(
    property(
        x = [0, 0, 2, 2],   # koordinat X polygon
        y = [4, 5, 5, 4],   # koordinat Y polygon
        name = "Lost Customers"
    )
)

# Tambahkan polygon untuk segmen "At Risk"
fig.add_trace(
    property (
        x = [0, 0, 2, 2],
        y = [0, 2, 2, 0],
        name = "At Risk",
    )
)

# Tambahkan polygon untuk segmen "Cannot Lose Them"
fig.add_trace(
    property (
        x = [0, 0, 2, 2],
        y = [2, 4, 4, 2],
        name = "Cannot Lose Them"
    )
)

# Tambahkan polygon untuk segmen "Champions"
fig.add_trace(
    property (
        x = [2, 2, 5, 5],
        y = [3, 5, 5, 3],
        name = "Champions"
    )
)

# Tambahkan polygon untuk segmen "About To Sleep"
fig.add_trace(
    property (
        x = [2, 2, 3.5, 3.5],
        y = [2, 3, 3, 2],
        name="About To Sleep"
    )
)

# Tambahkan polygon untuk segmen "Loyal"
fig.add_trace(
    property (
        x = [2, 2, 3.5, 3.5],
        y = [0, 2, 2, 0],
        name = "Loyal",
    )
)

# Tambahkan polygon untuk segmen "Potential Loyalist"
fig.add_trace(
    property (
        x = [3.5, 3.5, 5, 5],
        y = [1, 3, 3, 1],
        name = "Potential Loyalist",
    )
)

# Tambahkan polygon untuk segmen "Other"
fig.add_trace(
    property (
        x = [3.5, 3.5, 5, 5],
        y = [0, 1, 1, 0],
        name = "Other",
    )
)

# Menambahkan anotasi teks untuk setiap segmen (posisi disesuaikan agar sesuai area polygon)
annotation (x = 0.01, y = 1, name = "Lost Customers")
annotation (x = 0.01, y = 0.05, name = "At Risk")
annotation (x = 0.01, y = 0.5, name = "Cannot Lose Them")
annotation (x = 0.41, y = 1.0, name = "Champions")
annotation (x = 0.41, y = 0.50, name = "About To Sleep")
annotation (x = 0.41, y = 0.01, name = "Loyal")
annotation (x = 0.71, y = 0.51, name = "Potential Loyalist")
annotation (x = 0.71, y = 0.01, name = "Other")

# Mengatur layout figure: judul, warna background, skala axis, dan style tampilan
fig.update_layout(
    title = dict(text="<b>Segmentasi Pelanggan Berdasarkan Nilai RFM<b>",
                 font=dict(size=12, family="Arial", color="black")),
    plot_bgcolor = "white",  # warna background grafik
    xaxis = dict (showline = True, range = [0, 5],  rangemode = "nonnegative"),  # batas X-axis
    yaxis = dict (showline = True, range = [0, 5], rangemode = "nonnegative",
                  tickmode="array", tickvals=[1, 2, 3, 4, 5]),  # batas Y-axis dan label
    showlegend = False  # sembunyikan legend karena teks sudah ditampilkan melalui anotasi
)

# Tambahkan anotasi pembuat di bagian bawah figure
fig.add_annotation(
    text="Created By AL.",
    xref="paper", yref="paper", x=1, y=-.2,
    showarrow=False,
    font=dict(size=10, color="gray", family="Arial")
)

# Tampilkan visualisasi
fig.show()

In [48]:
# Mengurutkan data berdasarkan nilai total_monetary dari terbesar ke terkecil,
# kemudian mereset index agar urutan barunya rapi.
data_rfm_summary = data_rfm_summary.sort_values(by="total_monetary", ascending=False).reset_index()

# Menghitung persentase kumulatif kontribusi monetary dari setiap segmen
# Rumus: (cumulative sum total_monetary / total keseluruhan monetary) * 100
data_rfm_summary["Cumulative_%"] = data_rfm_summary["total_monetary"].cumsum() / data_rfm_summary["total_monetary"].sum() * 100

# Menampilkan 5 baris pertama hasil perhitungan untuk pengecekan
display(data_rfm_summary.head())

,index,Segment,customer_count,total_monetary,total_monetary_percent,total_monetary_scaling,Cumulative_%
0,3,Champions,11432,3607892.80,27.29,1.000000,27.288079
1,1,At Risk,19143,2830407.45,21.41,0.778986,48.695694
2,2,Cannot Lose Them,7290,2001949.19,15.14,0.543482,63.837315
3,5,Loyal,7524,1891409.09,14.31,0.512060,78.142873
4,7,Potential Loyalist,32849,1817609.29,13.75,0.491081,91.890251


In [49]:
def color_gradient(n_colors=5, tone="rocket_r"):
    """
    Menghasilkan gradasi warna atau palet warna kategorikal untuk Plotly.
    Parameter:
    - n_colors: jumlah warna yang ingin dibuat
    - tone: nama palet seaborn, contoh: 'rocket', 'viridis', 'Dark2', 'crest', 'mako', dll.
    """
    try:
        # Mencoba menggunakan palet bertipe kontinu (sequential) sebagai colormap
        cmap = sns.color_palette(tone, as_cmap=True)

        # Membuat posisi warna merata dari 0 hingga 1 untuk menghasilkan gradasi
        positions = np.linspace(0, 1, n_colors)

        # Mengonversi nilai colormap menjadi warna hex
        colors = [sns.utils.rgb2hex(cmap(p)) for p in positions]

    except Exception:
        # Jika terjadi error, berarti paletnya bukan bertipe kontinu
        # Maka gunakan palet diskrit (qualitative) standar dari seaborn
        colors = sns.color_palette(tone, n_colors).as_hex()

    # Mengembalikan daftar warna dalam format hex
    return colors

In [77]:
# Membuat figure kosong sebagai wadah visualisasi
fig = go.Figure()

# Menambahkan bar chart untuk menampilkan nilai total monetary tiap segmen
fig.add_trace(
    go.Bar (
        # Sumbu-X berisi nama segmen pelanggan
        x = data_rfm_summary["Segment"].astype(str),

        # Sumbu-Y berisi nilai total monetary (dibulatkan 2 desimal)
        y = data_rfm_summary["total_monetary"].round(2),

        name = "Total Monetary",

        # Menghasilkan warna otomatis dengan gradasi menggunakan fungsi color_gradient
        marker_color=color_gradient(n_colors=len(data_rfm_summary), tone="crest"),

        # Template teks ketika pengguna meng-hover bar
        hovertemplate="<b>%{x}</b><br>Total Monetary: $%{y: ,.2f}<extra></extra>"
    )
)

# Menambahkan line chart untuk menampilkan persentase kumulatif (Pareto)
fig.add_trace(
    go.Scatter (
        # Sumbu-X berisi segmen pelanggan
        x = data_rfm_summary["Segment"].astype(str),

        # Sumbu-Y menggunakan nilai persentase kumulatif
        y = data_rfm_summary["Cumulative_%"],

        name = "Cumulative %",

        # Line + marker + text untuk tampilan lebih informatif
        mode="lines+markers+text",

        # Menggunakan sumbu Y2 (kanan)
        yaxis = "y2",

        # Warna marker dan garis
        marker_color = "rgb(255,160,122)",
        line=dict(color="rgb(192, 57, 43)", width=3),

        # Template hover ketika cursor berada pada titik line chart
        hovertemplate="<b>%{x}</b><br>Cumulative: %{y:.1f}%<extra></extra>"
    )
)

# Mengatur layout untuk mempercantik tampilan chart
fig.update_layout(
    title = dict(
        text="<b>Bagan Pareto Segmen Pelanggan Berdasarkan Nilai Moneter<b>",  # Judul grafik
        font=dict(size=12, family="Arial", color="black")
    ),

    plot_bgcolor = "white",  # Warna background chart menjadi putih

    # Konfigurasi sumbu Y untuk bar chart (kiri)
    yaxis = dict(
        side = "left",
        showgrid = False,
        zeroline = True,
        showline = False,
        showticklabels = True
    ),

    # Konfigurasi sumbu Y2 untuk line chart (kanan)
    yaxis2 = dict(
        side = "right",
        overlaying = "y",
        showgrid = False,
        zeroline = False,
        showline = False,
        showticklabels = True,
        ticksuffix="%",      # Menambahkan tanda persen
        range=[0, 100]       # Rentang 0–100%
    ),

    # Konfigurasi sumbu X
    xaxis = dict(
        showline = True,
        linecolor = "rgb(204, 204, 204)",
        linewidth = 2
    ),

    # Menambahkan teks anotasi di bawah grafik
    annotations=[
        dict(
            text="Created By AL.",
            xref="paper", yref="paper",
            x=1, y=-.45,
            showarrow=False,
            font=dict(size=10, color="gray", family="Arial")
        )
    ],

    showlegend=False  # Menyembunyikan legend untuk tampilan bersih
)

# Anotasi tambahan di bagian bawah plot
fig.add_annotation(
    text="Created By AL.",
    xref="paper", yref="paper",
    x=1, y=-.2,
    showarrow=False,
    font=dict(size=10, color="gray")
)

# Menampilkan grafik
fig.show()

**Insight:**

- **Champions** dan **Potential Loyalists** adalah segmen yang paling berharga, masing-masing berkontribusi **6,62%** dan **21,59%** dari total pendapatan. Kelompok ini sebaiknya diprioritaskan untuk retensi dan *engagement*.

- **Other** memiliki basis pelanggan terbesar (77.803 pelanggan), berkontribusi **17,64%** dari pendapatan, segmen yang menjanjikan untuk analisis dan pembinaan yang lebih mendalam.

- Segmen seperti **Lost Customers**, **About To Sleep** dan **At Risk** memiliki nilai moneter yang sangat rendah (di bawah 2%), mungkin memerlukan strategi reaktivasi.

- **Cannot Lose Them** dan **Loyal** adalah kontributor yang stabil dan konsisten, ideal untuk manajemen hubungan yang berkelanjutan.

### Analisis Operasi E-commerce

#### Analisis Deret Waktu

##### GMV Harian

In [51]:
# Menggabungkan tabel orders dengan order items untuk mendapatkan detail produk, dan memfilter hanya pesanan yang sudah delivered
data_orders_timeseries_daily = df_orders.merge(df_order_items_, how="left", on="order_id")\
                                        .query('order_status == "delivered"')

# Mengambil hanya bagian tanggal dari tanggal delivered untuk agregasi harian
data_orders_timeseries_daily["date"] = data_orders_timeseries_daily["order_delivered_customer_date"].dt.to_period("D")

# Mengagregasi metrik harian: jumlah order, total produk terjual, total nilai penjualan
data_orders_timeseries_daily = (
    data_orders_timeseries_daily
    .groupby("date")
    .agg({
        "order_id": "count",          # jumlah order harian
        "product_counts": "sum",      # total produk terjual per hari
        "total_price": "sum",         # total nilai penjualan per hari
    })
    .query("date.isna() == False")   # menghapus baris dengan tanggal yang hilang
    .reset_index()
)

# Menghitung rata-rata bergerak 7 hari untuk total nilai penjualan
data_orders_timeseries_daily["7d_moving_average"] = (
    data_orders_timeseries_daily.sort_values(by="date")[["total_price"]]
    .transform(lambda x: round(x.rolling(7).mean(), 2))
)

# Menghitung rata-rata bergerak 30 hari untuk total nilai penjualan
data_orders_timeseries_daily["30d_moving_average"] = (
    data_orders_timeseries_daily.sort_values(by="date")[["total_price"]]
    .transform(lambda x: round(x.rolling(30).mean(), 2))
)

In [78]:
fig = go.Figure()

# Menambahkan scatter plot untuk GMV harian (total_price)
fig.add_trace(
    go.Scatter(
        x=data_orders_timeseries_daily["date"].astype(str),
        y=data_orders_timeseries_daily["total_price"],
        mode="markers",                      # ditampilkan sebagai titik
        name="GMV",                          # label legenda
        marker_color="#da3644",              # warna titik
    )
)

# Menambahkan garis rata-rata bergerak 7 hari
fig.add_trace(
    go.Scatter(
        x=data_orders_timeseries_daily["date"].astype(str),
        y=data_orders_timeseries_daily["7d_moving_average"],
        mode="lines",                        # ditampilkan sebagai garis
        name="7-D Moving Average",           # label legenda
        marker_color="#03b6fc",              # warna garis
    )
)

# Menambahkan garis rata-rata bergerak 30 hari
fig.add_trace(
    go.Scatter(
        x=data_orders_timeseries_daily["date"].astype(str),
        y=data_orders_timeseries_daily["30d_moving_average"],
        mode="lines",                        # ditampilkan sebagai garis
        name="30-D Moving Average",          # label legenda
        marker_color="#1de02d",              # warna garis
    )
)

# Pengaturan layout untuk tampilan grafik
fig.update_layout(
    title=dict(
        text="<b>Pendapatan Harian<b>",          # judul grafik
        font=dict(size=12, family="Arial", color="black")
    ),
    plot_bgcolor="white",                    # warna background grafik
    xaxis=dict(
        showline=True,                       # menampilkan garis sumbu X
        linecolor="rgb(204, 204, 204)",      # warna garis sumbu X
        linewidth=1.5,                       # ketebalan garis sumbu X
    ),
    yaxis=dict(showticklabels=True),         # menampilkan label pada sumbu Y
    legend=dict(orientation="h"),            # posisi legenda horizontal
    hovermode="x unified",                   # hover menyatu di sepanjang sumbu X
    showlegend=True                          # menampilkan legenda
)

# Menambahkan anotasi pada bagian bawah grafik
fig.add_annotation(
    text="Created By AL.",
    xref="paper", yref="paper",
    x=1, y=-0.2,
    showarrow=False,
    font=dict(size=10, color="gray", family="Arial")
)

fig.show()

**Insight:**

- Nilai GMV harian berfluktuasi selama periode tersebut, yang dapat berkisar antara 0 hingga lebih dari 60.000. Namun, tren keseluruhannya meningkat.

- Salah satu titik abnormal di sini adalah dari Oktober 2016 hingga Januari 2017 dan dari Oktober 2018, GMV tampaknya mendatar sebelum 5.000.

##### GMV Bulanan

In [53]:
# Menggabungkan orders dengan order items untuk mendapatkan detail per produk dan memfilter hanya pesanan yang berstatus delivered
data_orders_timeseries_monthly = df_orders.merge(df_order_items_, how="left", on="order_id")\
                                         .query('order_status == "delivered"')

# Mengekstrak periode tahun-bulan dari tanggal delivered untuk agregasi bulanan
data_orders_timeseries_monthly["year_month"] = data_orders_timeseries_monthly["order_delivered_customer_date"].dt.to_period("M")

# Mengagregasi metrik bulanan: jumlah pesanan, total produk, total nilai penjualan
data_orders_timeseries_monthly = (
    data_orders_timeseries_monthly
    .groupby("year_month")
    .agg({
        "order_id": "count",          # jumlah pesanan bulanan
        "product_counts": "sum",      # total produk terjual bulanan
        "total_price": "sum",         # total nilai penjualan bulanan
    })
    .query("year_month.isna() == False")  # menghapus baris dengan tanggal kosong
    .rename(columns={"order_id": "orders_count"})  # mengganti nama kolom agar lebih jelas
    .reset_index()
)

# Menghitung rata-rata bergerak 3 bulan dari total nilai penjualan
data_orders_timeseries_monthly["moving_average"] = (
    data_orders_timeseries_monthly.sort_values(by="year_month")[["total_price"]]
    .transform(lambda x: round(x.rolling(3).mean(), 2))
    .fillna(0)  # mengisi nilai NaN untuk 2 bulan pertama
)

# Menampilkan 5 baris pertama dari time series bulanan beserta moving average
display(data_orders_timeseries_monthly.head())

,year_month,orders_count,product_counts,total_price,moving_average
0,2016-10,216,241.0,29874.44,0.00
1,2016-11,61,72.0,9837.68,0.00
2,2016-12,4,4.0,758.86,13490.33
3,2017-01,294,326.0,33599.12,14731.89
4,2017-02,1410,1565.0,198909.29,77755.76


In [79]:
# Membuat figure kosong untuk visualisasi
fig = go.Figure()

# Menambahkan bar chart untuk jumlah pesanan per bulan
fig.add_trace(
    go.Bar (
        x = data_orders_timeseries_monthly["year_month"].astype(str),
        y = data_orders_timeseries_monthly["orders_count"],
        name = "Jumlah Pesanan",      # jumlah pesanan
        marker_color = "rgb(64,224,208)",
    )
)

# Menambahkan line chart untuk GMV per bulan (menggunakan axis kedua)
fig.add_trace(
    go.Scatter (
        x = data_orders_timeseries_monthly["year_month"].astype(str),
        y = data_orders_timeseries_monthly["total_price"],
        name = "GMV",                   # total nilai penjualan bulanan
        yaxis = "y2",                   # menggunakan sumbu Y kedua
        marker_color = "rgb(255,160,122)",
        mode="lines+markers"
    )
)

# Mengatur layout keseluruhan chart
fig.update_layout (
    title=dict(
        text="<b>GMV Bulanan<b>",       # judul chart
        font=dict(size=12, family="Arial", color="black")
    ),
    plot_bgcolor = "white",             # warna background chart
    xaxis = dict (
        showline = True,                # menampilkan garis sumbu X
        showgrid = False,               # menyembunyikan grid vertical
        linecolor = "rgb(204, 204, 204)",
        linewidth = 1.5,
    ),
    yaxis = dict (
        side = "left",                  # sumbu Y kiri
        showgrid = False,               # grid horizontal dimatikan
        zeroline = True,                # garis nol ditampilkan
        showline = False,               # garis sumbu tidak ditampilkan
        showticklabels = False,         # label nilai Y kiri disembunyikan
    ),
    yaxis2 = dict (
        side = "right",                 # sumbu Y kanan
        overlaying = "y",               # menumpuk di atas sumbu Y pertama
        showgrid = False,
        zeroline = False,
        showline = False,
        showticklabels = False,
    ),
    legend = dict (
        orientation="h",                # orientasi legend horizontal
    ),
    hovermode = "x unified"             # tooltip muncul seragam berdasarkan X
)

# Menambahkan catatan pembuat grafik
fig.add_annotation(
    text="Created By AL.",
    xref="paper", yref="paper",
    x=1, y=-0.2,
    showarrow=False,
    font=dict(size=10, color="gray", family="Arial")
)

# Menampilkan chart
fig.show()

**Insight:**

* **Tren pertumbuhan**: GMV dan jumlah pesanan terus meningkat dari akhir 2016 hingga pertengahan 2018 → pertumbuhan bisnis yang stabil.

* **Puncak**: GMV mencapai puncaknya sekitar Juli 2018 → periode penjualan paling efektif.

* **Penurunan tajam**: GMV dan pesanan turun tajam pada Oktober 2018, dengan GMV yang semakin menurun → selidiki penyebabnya (pasar, produk, operasional, dll).

##### Model Prediksi Penjualan

In [55]:
# Bekerja menggunakan salinan dari data time series pesanan bulanan
data_orders_timeseries = data_orders_timeseries_monthly.copy()

# Menghitung selisih month-over-month dari total penjualan
data_orders_timeseries["total_price_diff"] = data_orders_timeseries["total_price"].diff()

# Menambahkan indeks bulan (1, 2, 3, ...) untuk merepresentasikan waktu
data_orders_timeseries["month"] = data_orders_timeseries.index + 1

# Menyiapkan fitur untuk regresi time series
# Dimulai dengan kolom selisih penjualan (diff)
sales_data = data_orders_timeseries[["total_price_diff"]].copy(deep=True).fillna(0)

# Menyimpan total penjualan aktual untuk evaluasi nanti
sales_actual = data_orders_timeseries["total_price"].to_list()

# Membuat fitur lag untuk 12 bulan sebelumnya
for month in range(1, 13):
    col_name = "month_" + str(month)
    sales_data[col_name] = sales_data["total_price_diff"].shift(month)  # selisih bulan-bulan sebelumnya (lag)

# Menghapus baris yang memiliki nilai NaN akibat proses lag dan reset index
sales_data = sales_data.dropna().reset_index(drop=True)

display(sales_data.head())

,total_price_diff,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,77296.91,44738.43,80528.94,24221.41,-89540.87,251633.22,-67198.98,136976.92,165310.17,32840.26,-9078.82,-20036.76,0.00
1,-5990.45,77296.91,44738.43,80528.94,24221.41,-89540.87,251633.22,-67198.98,136976.92,165310.17,32840.26,-9078.82,-20036.76
2,293418.08,-5990.45,77296.91,44738.43,80528.94,24221.41,-89540.87,251633.22,-67198.98,136976.92,165310.17,32840.26,-9078.82
3,-93874.78,293418.08,-5990.45,77296.91,44738.43,80528.94,24221.41,-89540.87,251633.22,-67198.98,136976.92,165310.17,32840.26
4,-101147.56,-93874.78,293418.08,-5990.45,77296.91,44738.43,80528.94,24221.41,-89540.87,251633.22,-67198.98,136976.92,165310.17


In [56]:
# Membagi data menjadi training (10 bulan pertama) dan test set (bulan-bulan berikutnya)
train_data = sales_data[:10]
test_data = sales_data[10:]

# Melakukan penskalaan fitur ke rentang [-1, 1] menggunakan MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(train_data)

# Menerapkan penskalaan pada dataset training dan test
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

# Memisahkan fitur (lag bulan sebelumnya) dan target (selisih bulan saat ini)
X_train, y_train = train_data[:, 1:], train_data[:, 0:1].ravel()  # target adalah kolom pertama
X_test, y_test = test_data[:, 1:], test_data[:, 0:1].ravel()

# Mendefinisikan pipeline regresi menggunakan spline transformation + Ridge regression
model = make_pipeline(
    SplineTransformer(
        knots=np.linspace(0, np.pi**2 + 5, 36).reshape(3, 12),  # titik knot spline
        degree=2,                                               # spline kuadratik
        extrapolation="periodic"                                # menangani data di luar jangkauan secara periodik
    ),
    Ridge(alpha=1e-8, max_iter=10000, random_state=42)         # Regresi Ridge
)

# Melatih model menggunakan data training
model.fit(X_train, y_train)

# Memprediksi nilai selisih untuk data test
y_pred = model.predict(X_test).reshape(-1, 1)

# Menggabungkan prediksi dengan fitur lag untuk melakukan inverse scaling
y_pred = np.concatenate([y_pred, X_test], axis=1)

# Mengembalikan nilai ke skala asli (inverse transform)
sales_pred = scaler.inverse_transform(y_pred)

# Menambahkan selisih prediksi dengan penjualan sebelumnya untuk mendapatkan prediksi total penjualan
result = []
for index in range(len(sales_pred)):
    if (sales_pred[index][0] + sales_actual[index]) < 0:
        result.append(0)  # menghindari hasil penjualan negatif
    else:
        result.append(sales_pred[index][0] + sales_actual[index])

# Menghitung skor R^2 untuk 3 bulan terakhir
accuracy = r2_score(sales_actual[-3:], result)
print(f"Akurasinya adalah {accuracy*100:.2f}%")

Akurasinya adalah 73.13%


In [80]:
# Membuat figure kosong untuk visualisasi
fig = go.Figure()

# Menambahkan trace untuk data penjualan aktual (Actual Sales)
fig.add_trace(
    go.Scatter (
        x = data_orders_timeseries["year_month"].astype(str),  # sumbu X: bulan dalam format string
        y = data_orders_timeseries["total_price"],             # sumbu Y: nilai penjualan aktual
        name = "Penjualan Aktual",                                 # label legend
        marker_color = "rgb(255,160,122)",                     # warna garis dan marker
        mode = "lines+markers"                                 # tampilkan garis dan titik
    )
)

# Menambahkan trace untuk data prediksi penjualan (Predicted Sales)
fig.add_trace(
    go.Scatter (
        x = data_orders_timeseries["year_month"].astype(str).to_list()[-3:],  # hanya 3 bulan terakhir
        y = result,                                                            # nilai prediksi
        name = "Prediksi Penjualan",                                              # label legend
        marker_color = "lightseagreen",                                        # warna garis dan marker
        mode = "lines+markers"                                                 # tampilkan garis dan titik
    )
)

# Mengatur tampilan layout grafik
fig.update_layout (
    title=dict(
        text="<b>Aktual vs Prediksi GMV<b>",               # judul grafik
        font=dict(size=12, family="Arial", color="black")   # pengaturan font judul
    ),
    plot_bgcolor = "white",                                 # warna background plot
    xaxis = dict (
        showline = True,                                     # tampilkan garis sumbu X
        showgrid = False,                                    # hilangkan grid
        linecolor = "rgb(204, 204, 204)",                    # warna garis sumbu X
        linewidth = 1.5,                                     # ketebalan garis
    ),
    legend = dict (
        orientation="h",                                     # legend horizontal
    ),
    hovermode = "x unified"                                  # hover menyatu per posisi X
)

# Menambahkan catatan pembuat grafik
fig.add_annotation(
    text="Created By AL.",                           # teks catatan
    xref="paper", yref="paper",                              # referensi posisi relatif terhadap canvas
    x=1, y=-0.2,                                              # posisi teks di luar grafik
    showarrow=False,                                          # tanpa panah
    font=dict(size=10, color="gray", family="Arial")          # pengaturan font
)

# Menampilkan grafik
fig.show()

**Insight:**

* **GMV Aktual** tumbuh stabil dari akhir 2016 hingga pertengahan 2018, mencapai puncaknya pada Juli 2018, lalu turun tajam.

* **GMV Prediksi** hanya muncul dari Juli hingga Oktober 2018, menunjukkan deviasi yang besar, terutama pada puncaknya.

* Model prediksi gagal menangkap tren nyata → perlu peningkatan akurasi dan respons terhadap perubahan pasar.

#### Kepuasan Pelanggan

In [58]:
# Mengagregasi skor ulasan pada level pesanan
data_review = (
    df_order_reviews[["order_id", "review_score"]]  # memilih kolom yang relevan
    .groupby("order_id")                            # mengelompokkan berdasarkan setiap order
    .agg({"review_score": "mean"})                  # menghitung rata-rata skor ulasan per order
    .reset_index()                                  # reset index agar DataFrame rapi
)

display(data_review.head())

,order_id,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,5.0
1,00018f77f2f0320c557190d7a144bdd3,4.0
2,000229ec398224ef6ca0657da4fc703e,5.0
3,00024acbcdf0a6daa1e931b038114c75,4.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,5.0


In [59]:
# Membuat dataset pesanan yang komprehensif dengan menggabungkan beberapa sumber
data_orders_ = (
    df_orders.copy(deep=True)  # membuat salinan mendalam dari data pesanan
    # Menggabungkan rata-rata skor ulasan per pesanan
    .merge(data_review, how="left", on="order_id")
    # Menggabungkan informasi item pesanan: seller_id dan freight_value
    .merge(df_order_items[["order_id", "seller_id", "freight_value"]], how="left", on="order_id")
    # Menggabungkan informasi penjual: kota dan provinsi
    .merge(df_sellers[["seller_id", "seller_city", "seller_state"]], how="left", on="seller_id")
    # Menggabungkan informasi pelanggan: kota dan provinsi
    .merge(df_customers[["customer_id", "customer_city", "customer_state"]], how="left", on="customer_id")
    # Menghapus baris duplikat dengan menyimpan kemunculan pertama
    .drop_duplicates(keep="first")
    # Mengatur ulang indeks agar DataFrame lebih rapi
    .reset_index(drop=True)
)

# Mengisi skor ulasan yang hilang dengan 0 (untuk pesanan tanpa ulasan)
data_orders_["review_score"].fillna(0, inplace=True)

display(data_orders_.head())

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_score,seller_id,freight_value,seller_city,seller_state,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,4.0,3504c0cb71d7fa48d967e0e4c94d59d9,8.72,maua,SP,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,4.0,289cdb325fb7e7f891c38608bf9e0962,22.76,belo horizonte,SP,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,5.0,4869f7a5dfa277a7dca6462dcf3b52b2,19.22,guariba,SP,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,5.0,66922902710d126a0e7d26b0e3805106,27.20,belo horizonte,MG,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,5.0,2c9e548be18521d1c43cde1c582c6de8,8.72,mogi das cruzes,SP,santo andre,SP


In [60]:
# Ekstrak tahun-bulan dari timestamp pembelian pesanan untuk analisis bulanan
data_orders_["year_month"] = data_orders_["order_purchase_timestamp"].dt.to_period("M")

# Hitung waktu (dalam menit) dari pembelian pesanan hingga persetujuan
data_orders_["approved_mins"] = (
    (data_orders_["order_approved_at"] - data_orders_["order_purchase_timestamp"])
    .dt.total_seconds() / 60
).fillna(0).astype(int)

# Hitung waktu (dalam hari) dari persetujuan pesanan hingga pengiriman ke kurir
data_orders_["to_carrier_days"] = (
    (data_orders_["order_delivered_carrier_date"] - data_orders_["order_approved_at"])
    .dt.days
).fillna(0).astype(int)

# Hitung waktu (dalam hari) dari pengiriman kurir hingga pesanan diterima pelanggan
data_orders_["to_customer_days"] = (
    (data_orders_["order_delivered_customer_date"] - data_orders_["order_delivered_carrier_date"])
    .dt.days
).fillna(0).astype(int)

# Total waktu pengiriman (dari persetujuan hingga pesanan diterima pelanggan)
data_orders_["delivery_days"] = data_orders_["to_carrier_days"] + data_orders_["to_customer_days"]

# Tandai pesanan yang dikirim lebih lambat dari tanggal estimasi pengiriman
data_orders_["is_late"] = np.where(
    data_orders_["order_estimated_delivery_date"] <= data_orders_["order_delivered_customer_date"],
    True, False)

display(data_orders_.head())

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_score,seller_id,freight_value,seller_city,seller_state,customer_city,customer_state,year_month,approved_mins,to_carrier_days,to_customer_days,delivery_days,is_late
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,4.0,3504c0cb71d7fa48d967e0e4c94d59d9,8.72,maua,SP,sao paulo,SP,2017-10,10,2,6,8,False
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,4.0,289cdb325fb7e7f891c38608bf9e0962,22.76,belo horizonte,SP,barreiras,BA,2018-07,1842,0,12,12,False
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,5.0,4869f7a5dfa277a7dca6462dcf3b52b2,19.22,guariba,SP,vianopolis,GO,2018-08,16,0,9,9,False
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,5.0,66922902710d126a0e7d26b0e3805106,27.20,belo horizonte,MG,sao goncalo do amarante,RN,2017-11,17,3,9,12,False
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,5.0,2c9e548be18521d1c43cde1c582c6de8,8.72,mogi das cruzes,SP,santo andre,SP,2018-02,61,0,1,1,False


In [61]:
# Membuat daftar kolom yang relevan untuk analisis
columns_list = [
    "order_id",          # ID pesanan
    "customer_id",       # ID pelanggan
    "customer_city",     # Kota pelanggan
    "customer_state",    # Provinsi pelanggan
    "seller_id",         # ID penjual
    "seller_city",       # Kota penjual
    "seller_state",      # Provinsi penjual
    "order_status",      # Status pesanan
    "order_purchase_timestamp",  # Waktu pembelian pesanan
    "review_score",      # Skor ulasan
    "freight_value",     # Nilai ongkos kirim
    "year_month",        # Tahun-bulan pembelian
    "approved_mins",     # Waktu persetujuan (menit)
    "to_carrier_days",   # Waktu ke kurir (hari)
    "to_customer_days",  # Waktu ke pelanggan (hari)
    "delivery_days",     # Total waktu pengiriman (hari)
    "is_late"            # Penanda apakah pesanan terlambat
]

# Menyaring dataset agar hanya menyertakan kolom yang ada di daftar
data_orders_ = data_orders_[columns_list]

# Menampilkan 5 baris pertama dari dataset hasil penyaringan
display(data_orders_.head())

,order_id,customer_id,customer_city,customer_state,seller_id,seller_city,seller_state,order_status,order_purchase_timestamp,review_score,freight_value,year_month,approved_mins,to_carrier_days,to_customer_days,delivery_days,is_late
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,sao paulo,SP,3504c0cb71d7fa48d967e0e4c94d59d9,maua,SP,delivered,2017-10-02 10:56:33,4.0,8.72,2017-10,10,2,6,8,False
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,barreiras,BA,289cdb325fb7e7f891c38608bf9e0962,belo horizonte,SP,delivered,2018-07-24 20:41:37,4.0,22.76,2018-07,1842,0,12,12,False
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,vianopolis,GO,4869f7a5dfa277a7dca6462dcf3b52b2,guariba,SP,delivered,2018-08-08 08:38:49,5.0,19.22,2018-08,16,0,9,9,False
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,sao goncalo do amarante,RN,66922902710d126a0e7d26b0e3805106,belo horizonte,MG,delivered,2017-11-18 19:28:06,5.0,27.20,2017-11,17,3,9,12,False
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,santo andre,SP,2c9e548be18521d1c43cde1c582c6de8,mogi das cruzes,SP,delivered,2018-02-13 21:18:39,5.0,8.72,2018-02,61,0,1,1,False


In [81]:
# Membuat objek Figure dengan histogram
fig = go.Figure(
    go.Histogram(
        x = data_orders_["review_score"],       # Data skor ulasan pelanggan
        histfunc = "count",                     # Hitung jumlah pesanan per skor
        texttemplate = "%{y}",                  # Tampilkan jumlah pesanan di atas batang
        name = "Nilai Ulasan",                  # Nama grafik
        textfont_size = 12,                     # Ukuran font teks
        textposition = "outside",               # Posisi teks di luar batang
        hovertemplate = "Nilai Ulasan: %{x}<br>Jumlah Pesanan: %{y}",  # Tooltip saat hover
        xbins = dict (
            start = -1.0,                       # Nilai awal bin (mulai dari -1 untuk keamanan)
            end = 5.0,                          # Nilai akhir bin (skor ulasan maksimum = 5)
            size = 1.0                          # Ukuran bin = 1 (interval per skor)
        )
    )
)

# Mengatur layout grafik
fig.update_layout(
    title=dict(
        text="<b>Ulasan Pelanggan<b>",           # Judul grafik
        font=dict(size=12, family="Arial", color="black")  # Format font judul
    ),
    plot_bgcolor = "white",                     # Latar belakang grafik putih
    xaxis = dict(title = "Nilai Ulasan"),       # Label sumbu X
    yaxis = dict(showticklabels = False)        # Sumbu Y tanpa label angka
)

# Menambahkan anotasi (catatan tambahan) di bawah grafik
fig.add_annotation(
    text="Created By AL.",               # Teks anotasi
    xref="paper", yref="paper",                 # Referensi posisi relatif terhadap grafik
    x=1, y=-0.2,                                # Posisi anotasi
    showarrow=False,                            # Tidak menampilkan panah
    font=dict(size=10, color="gray", family="Arial")  # Format font anotasi
)

# Menampilkan grafik
fig.show()

**Insight:**

* Peringkat 5 & 4 mendominasi (masing-masing ~19.3K) → mayoritas pelanggan puas.

* Peringkat 1 juga tinggi (12.05K) → jelas merupakan segmen pelanggan yang tidak puas.

* Peringkat menengah (2 & 3) lebih rendah → umpan balik terpolarisasi.

* Peringkat 0 jarang (799) tetapi perlu diselidiki → mungkin mengindikasikan masalah sistem.

#### Rasio Penyelesaian Pengiriman

In [63]:
# Hitung jumlah pesanan untuk setiap status pesanan
data_orders_status = (
    data_orders_.groupby("order_status")["order_id"]
    .count()  # menghitung jumlah pesanan per status
    .reset_index()
)

# Hitung persentase setiap status pesanan terhadap total pesanan
data_orders_status["%ratio"] = data_orders_status["order_id"] / data_orders_status["order_id"].sum() * 100

# Tampilkan 5 baris pertama dari hasil
display(data_orders_status.head())

,order_status,order_id,%ratio
0,approved,2,0.001962
1,canceled,628,0.616170
2,created,5,0.004906
3,delivered,98937,97.073195
4,invoiced,318,0.312009


In [64]:
# Hitung total pengiriman (semua pesanan)
total_shipments = data_orders_["order_id"].nunique()

# Hitung total pengiriman yang dibatalkan
total_cancelled_shipments = data_orders_[data_orders_["order_status"] == "canceled"]["order_id"].nunique()

# Hitung dan cetak rasio pengiriman yang dibatalkan
print(f"Rasio pengiriman yang dibatalkan adalah {(total_cancelled_shipments / total_shipments) * 100:.2f}%")

Rasio pengiriman yang dibatalkan adalah 0.63%


In [65]:
# Hitung total pengiriman yang berhasil dikirim
total_delivered_shipments = data_orders_[data_orders_["order_status"] == "delivered"]["order_id"].nunique()

# Hitung total pengiriman yang tepat waktu (is_late == False)
total_on_time_delivered_shipments = data_orders_[
    (data_orders_["order_status"] == "delivered") & (data_orders_["is_late"] == False)
]["order_id"].nunique()

# Hitung dan cetak rasio pengiriman tepat waktu
print(f"Rasio pengiriman tepat waktu adalah {(total_on_time_delivered_shipments / total_delivered_shipments) * 100:.2f}%")

Rasio pengiriman tepat waktu adalah 91.89%


In [66]:
# Hitung rata-rata waktu persetujuan (dalam menit) per pesanan, tidak termasuk pesanan dengan waktu persetujuan = 0
avg_confirmed_mins = (
    data_orders_.query("approved_mins != 0")                    # menyaring pesanan dengan waktu persetujuan = 0
    .groupby("order_id")                                       # kelompokkan berdasarkan setiap pesanan
    .agg({"approved_mins": "mean"})["approved_mins"]           # hitung rata-rata waktu persetujuan per pesanan
    .mean()                                                    # rata-rata keseluruhan dari semua pesanan
)

# Hitung rata-rata waktu pengiriman (dalam hari) per pesanan, tidak termasuk pesanan dengan waktu pengiriman = 0
avg_delivery_days = (
    data_orders_.query("delivery_days != 0")                   # menyaring pesanan dengan waktu pengiriman = 0
    .groupby("order_id")                                       # kelompokkan berdasarkan setiap pesanan
    .agg({"delivery_days": "mean"})["delivery_days"]           # hitung rata-rata waktu pengiriman per pesanan
    .mean()                                                    # rata-rata keseluruhan dari semua pesanan
)

# Cetak hasil
print(f"Pada setiap pesanan, rata-rata membutuhkan sekitar {avg_confirmed_mins:.0f} menit agar pesanan disetujui.")
print(f"Pada setiap pesanan, rata-rata membutuhkan sekitar {avg_delivery_days:.0f} hari untuk dikirim ke pelanggan setelah disetujui.")

Pada setiap pesanan, rata-rata membutuhkan sekitar 633 menit agar pesanan disetujui.
Pada setiap pesanan, rata-rata membutuhkan sekitar 11 hari untuk dikirim ke pelanggan setelah disetujui.


In [67]:
# Mengagregasi jumlah pesanan bulanan berdasarkan status pesanan
data_orders_year_month = (
    data_orders_[["order_id", "year_month", "order_status"]]  # pilih kolom yang relevan
    .drop_duplicates()                                        # hapus pesanan duplikat
    .groupby(["year_month", "order_status"])                  # kelompokkan berdasarkan bulan dan status
    .agg(orders_count=("order_status", "count"))              # hitung jumlah pesanan per status
    .reset_index()
    # Ubah bentuk tabel agar satu baris per bulan dan satu kolom per status pesanan
    .pivot(index="year_month", columns="order_status", values="orders_count")
    .reset_index()                                            # atur ulang indeks agar DataFrame lebih rapi
    .fillna(0)                                                # isi nilai yang hilang dengan 0
)

# Tampilkan 5 baris pertama dari hasil agregasi
display(data_orders_year_month.head())

order_status,year_month,approved,canceled,created,delivered,invoiced,processing,shipped,unavailable
0,2016-09,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0
1,2016-10,0.0,24.0,0.0,265.0,18.0,2.0,8.0,7.0
2,2016-12,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2017-01,0.0,3.0,0.0,750.0,12.0,9.0,16.0,10.0
4,2017-02,1.0,17.0,0.0,1653.0,11.0,32.0,21.0,45.0


In [82]:
# Buat daftar bulan-tahun dari kolom year_month
year_month_list = data_orders_year_month["year_month"].astype(str).to_list()

# Buat daftar status pesanan unik
order_status_list = list(set(data_orders_["order_status"]))

# Inisialisasi objek Figure kosong
fig = go.Figure()

# Tambahkan garis (trace) untuk setiap status pesanan
for order_status in order_status_list:
    fig.add_trace(go.Scatter(
        x = year_month_list,                               # sumbu X = daftar bulan-tahun
        y = data_orders_year_month[order_status].to_list(),# sumbu Y = jumlah pesanan per status
        name = order_status,                               # nama garis sesuai status pesanan
        mode = "lines",                                    # mode visualisasi = garis
        line = dict(width = 0.5),                          # ketebalan garis
        stackgroup = "one"                                 # definisi grup tumpukan (stack group)
    ))

# Atur layout grafik
fig.update_layout(
    title=dict(
        text="<b>Pesanan Berdasarkan Status<b>",                     # judul grafik
        font=dict(size=12, family="Arial", color="black")  # format font judul
    ),
    plot_bgcolor = "white",                                # latar belakang grafik putih
    hovermode = "x unified"                                # tampilkan tooltip gabungan saat hover
)

# Tambahkan anotasi (catatan tambahan) di bawah grafik
fig.add_annotation(
    text="Created By AL.",                          # teks anotasi
    xref="paper", yref="paper",                            # referensi posisi relatif terhadap grafik
    x=1.1, y=-0.2,                                         # posisi anotasi
    showarrow=False,                                       # tidak menampilkan panah
    font=dict(size=10, color="gray", family="Arial")       # format font anotasi
)

# Tampilkan grafik
fig.show()

**Insight:**

* **Pesanan terkirim mendominasi** → sebagian besar pesanan berhasil diselesaikan.

* **Dibuat, diproses, disetujui, dikirim, ditagih, dibatalkan, tidak tersedia** → sangat sedikit pesanan dalam status ini, dampaknya minimal.

* **Tren dari waktu ke waktu**: Pesanan terkirim terus meningkat sejak awal 2017, mencapai puncaknya di awal 2018, kemudian sedikit menurun pada pertengahan 2018.

* Fokus pada mempertahankan tingkat pengiriman yang tinggi dan selidiki penyebab pesanan yang dibatalkan/tidak tersedia untuk meningkatkan efisiensi pemenuhan pesanan.

In [69]:
# Inisialisasi kolom baru untuk menyimpan total pesanan per bulan
data_orders_year_month["total_orders"] = 0

# Jumlahkan semua status pesanan untuk mendapatkan total pesanan per bulan
for status in set(data_orders_["order_status"]):
    data_orders_year_month["total_orders"] += data_orders_year_month[status]

# Hitung rata-rata pesanan bulanan
avg_orders_by_month = data_orders_year_month["total_orders"].mean()
print(f"Rata-rata, {avg_orders_by_month:.0f} pesanan dibuat per bulan.")

# Hitung total pesanan dalam dataset
total_orders = data_orders_year_month["total_orders"].sum()

# Hitung total hari yang tercakup dalam dataset
total_days = (data_orders_["order_purchase_timestamp"].max() - data_orders_["order_purchase_timestamp"].min()).days
print(f"Rata-rata, {total_orders/total_days:.0f} pesanan dibuat per hari.")

# Hitung total jam yang tercakup dan rata-rata pesanan per jam
total_hours = total_days * 24
print(f"Rata-rata, {total_orders/total_hours:.0f} pesanan dibuat per jam.")

Rata-rata, 3978 pesanan dibuat per bulan.
Rata-rata, 129 pesanan dibuat per hari.
Rata-rata, 5 pesanan dibuat per jam.


In [70]:
# Membuat DataFrame sementara dengan order_id dan timestamp pembelian
temp = (
    data_orders_[["order_id", "order_purchase_timestamp"]]
    .copy()
    .sort_values(by="order_purchase_timestamp")  # urutkan pesanan secara kronologis
    .drop_duplicates()                           # pastikan setiap pesanan dihitung sekali saja
)

# Hitung selisih waktu (dalam menit) antara pesanan yang berurutan
temp["order_purchase_diff"] = (
    temp["order_purchase_timestamp"].diff()      # selisih waktu antara pesanan berurutan
    .dt.total_seconds() / 60                     # ubah detik menjadi menit
).fillna(0).astype(int)                          # isi NaN pertama dengan 0 dan ubah ke integer

# Hitung rata-rata selisih waktu antar pesanan
avg_diff = temp["order_purchase_diff"].mean()

# Cetak hasil
print(f"Rata-rata, setiap {avg_diff:.0f} menit akan tercipta sebuah pesanan baru.")

Rata-rata, setiap 11 menit akan tercipta sebuah pesanan baru.


#### Evaluasi Penjual

In [71]:
# Mengagregasi metrik performa pengiriman berdasarkan rute (kota/provinsi penjual -> kota/provinsi pelanggan)
data_shipping = (
    data_orders_[[
        "order_id", "seller_city", "order_status", "seller_state",
        "customer_city", "customer_state", "freight_value",
        "to_carrier_days", "to_customer_days", "delivery_days"
    ]]
    .query('order_status == "delivered"')  # hanya mempertimbangkan pesanan yang berhasil dikirim
    .groupby(["seller_city", "seller_state", "customer_city", "customer_state"])
    .agg(
        orders_num_same_route=("order_id", "nunique"),  # jumlah pesanan unik pada rute yang sama
        avg_freight_value=("freight_value", "mean"),    # rata-rata biaya ongkos kirim
        avg_delivery_days=("delivery_days", "mean"),    # rata-rata hari pengiriman
    )
    .reset_index()
)

# Mendefinisikan tipe data yang diinginkan untuk metrik agregasi
obj_type = {
    "orders_num_same_route": "int",
    "avg_freight_value": "float",
    "avg_delivery_days": "int",
}

# Terapkan tipe data
data_shipping = data_shipping.astype(obj_type)

# Tampilkan 5 baris pertama dari hasil agregasi
display(data_shipping.head())

,seller_city,seller_state,customer_city,customer_state,orders_num_same_route,avg_freight_value,avg_delivery_days
0,abadia de goias,GO,sobral,CE,1,43.41,24
1,afonso claudio,ES,belem,PA,1,29.62,20
2,afonso claudio,ES,franca,SP,1,17.19,11
3,afonso claudio,ES,macae,RJ,1,15.56,6
4,afonso claudio,ES,niteroi,RJ,1,17.43,7


In [72]:
# Mengagregasi metrik performa penjual
data_sellers = (
    data_orders_[[
        "order_id", "seller_id", "seller_city", "seller_state",
        "order_status", "is_late", "review_score", "approved_mins"
    ]]
    .drop_duplicates()  # hapus baris duplikat agar agregasi bersih
    .groupby(["seller_id", "seller_city", "seller_state"])
    .apply(lambda x: pd.Series({
        'total_orders': x['order_id'].nunique(),  # total pesanan yang ditangani penjual
        'delivered_orders': x.query("order_status == 'delivered'")['order_id'].nunique(),  # pesanan yang berhasil dikirim
        'cancelled_orders': x.query("order_status == 'canceled'")['order_id'].nunique(),   # pesanan yang dibatalkan
        'late_delivery': x.query("is_late == True")['order_id'].nunique(),  # pesanan yang terlambat dikirim
        'avg_approved_mins': x['approved_mins'].mean(),  # rata-rata waktu persetujuan dalam menit
        'review_score': x.query("review_score != 0")['review_score'].mean(),  # rata-rata skor ulasan (tidak termasuk 0)
    }))
    # Hitung rasio pesanan terkirim dan dibatalkan dalam bentuk persentase
    .assign(
        delivered_ratio=lambda x: round(x["delivered_orders"] / x["total_orders"] * 100, 2),
        cancelled_ratio=lambda x: round(x["cancelled_orders"] / x["total_orders"] * 100, 2),
    )
    .reset_index()  # atur ulang indeks agar DataFrame lebih rapi
)

# Tampilkan 5 baris pertama dari hasil agregasi
display(data_sellers.head())

,seller_id,seller_city,seller_state,total_orders,delivered_orders,cancelled_orders,late_delivery,avg_approved_mins,review_score,delivered_ratio,cancelled_ratio
0,0015a82c2db000af6aaaf3ae2ecb0532,santo andre,SP,3.0,3.0,0.0,0.0,800.666667,3.666667,100.00,0.0
1,001cca7ae9ae17fb1caed9dfb1094831,cariacica,ES,200.0,195.0,0.0,13.0,591.290000,3.984772,97.50,0.0
2,001e6ad469a905060d959994f1b41e4f,sao goncalo,RJ,1.0,0.0,1.0,0.0,14.000000,1.000000,0.00,100.0
3,002100f778ceb8431b7a1020ff7ab48f,franca,SP,51.0,50.0,0.0,9.0,1376.313725,3.901961,98.04,0.0
4,003554e2dce176b5555353e4f3555ac8,goiania,GO,1.0,1.0,0.0,0.0,18.000000,5.000000,100.00,0.0


In [73]:
# Mengagregasi penjualan produk per penjual dengan terjemahan kategori
data_items = (
    df_order_items
    # Gabungkan nama kategori produk
    .merge(df_products[["product_id", "product_category_name"]], how="left", on="product_id")
    # Gabungkan nama kategori produk yang sudah diterjemahkan
    .merge(df_product_category_name_translation, how="left", on="product_category_name")
    # Kelompokkan berdasarkan penjual, produk, dan kategori produk (bahasa Inggris)
    .groupby(["seller_id", "product_id", "product_category_name_english"])
    .agg(
        product_count=("product_id", "count")  # hitung jumlah produk yang terjual oleh penjual
    )
    # Urutkan berdasarkan jumlah produk terjual (descending)
    .sort_values(by="product_count", ascending=False)
    .reset_index()
)

# Tampilkan 5 baris pertama dari hasil agregasi
display(data_items.head())

,seller_id,product_id,product_category_name_english,product_count
0,955fee9216a65b617aa5c0531780ce60,aca2eb7d00ea1a7b8ebd4e68314663af,furniture_decor,527
1,1f50f920176fa81dab994f9023523100,422879e10f46682990de24d770e7f83d,garden_tools,484
2,4a3ca9315b744ce9f8e9374361493884,99a4788cb24856965c36a24e339b6058,bed_bath_table,482
3,1f50f920176fa81dab994f9023523100,389d119b48cf3043d311335e499d9c6b,garden_tools,392
4,1f50f920176fa81dab994f9023523100,368c6c730842d78016ad823897a372db,garden_tools,388


In [74]:
def suggest(city: str, state: str, product_category, limit=10, product_id=None):
    """
    Menyarankan penjual berdasarkan kota dan provinsi pelanggan serta pencarian kategori produk
    (atau produk spesifik).

    Parameter:
    - city: kota pelanggan
    - state: provinsi pelanggan
    - product_category: kategori produk (bahasa Inggris)
    - limit: jumlah penjual yang disarankan untuk ditampilkan (default=10)
    - product_id: produk spesifik untuk filter (opsional)

    Return:
    - DataFrame penjual yang disarankan dengan metrik detail
    """

    # Filter penjual yang menjual produk spesifik atau kategori produk tertentu
    if product_id is not None:
        df_product = data_items[
            (data_items["product_id"] == product_id) &
            (data_items["product_category_name_english"] == product_category)
        ]
    else:
        df_product = data_items[
            data_items["product_category_name_english"] == product_category
        ]

    # Agregasi produk yang dijual per penjual
    df_product = df_product.groupby("seller_id").apply(
        lambda x: pd.Series({
            "products_sold": x[["product_id", "product_count"]].to_dict("records")
        })
    )

    # Gabungkan dengan metrik performa penjual
    df = df_product.merge(data_sellers, how="left", on="seller_id")

    # Filter penjual yang dapat mengirim ke kota/provinsi pelanggan
    df_location = data_shipping[
        (data_shipping["customer_city"] == city) &
        (data_shipping["customer_state"] == state)
    ]
    df = df.merge(df_location, how="inner", on=["seller_city", "seller_state"])

    # Urutkan penjual berdasarkan:
    # 1) rasio pesanan terkirim tertinggi
    # 2) rata-rata hari pengiriman terendah
    # 3) rata-rata waktu persetujuan terendah
    df = df.sort_values(
        by=["delivered_ratio", "avg_delivery_days", "avg_approved_mins"],
        ascending=[False, True, True]
    ).reset_index(drop=True)

    # Pilih kolom relevan untuk ditampilkan
    df = df[[
        "seller_id", "seller_city", "seller_state", "customer_city", "customer_state",
        "products_sold", "total_orders", "delivered_orders", "cancelled_orders", "late_delivery",
        "avg_approved_mins", "review_score", "delivered_ratio", "cancelled_ratio",
        "orders_num_same_route", "avg_freight_value", "avg_delivery_days"
    ]]

    return df

# Contoh penggunaan fungsi untuk menyarankan penjual kategori "garden_tools" di kota Sao Goncalo, RJ
suggest(product_category="garden_tools", city="sao goncalo", state="RJ").head()

,seller_id,seller_city,seller_state,customer_city,customer_state,products_sold,total_orders,delivered_orders,cancelled_orders,late_delivery,avg_approved_mins,review_score,delivered_ratio,cancelled_ratio,orders_num_same_route,avg_freight_value,avg_delivery_days
0,0a198e95d32b1be2da9424c962a6ebfa,contagem,MG,sao goncalo,RJ,"[{'product_id': 'f0788219c3d63c6183bbca7699ca9608', 'product_count': 1}]",1.0,1.0,0.0,0.0,27.000000,5.00,100.0,0.0,3,28.963333,4
1,2aa3443d7bf9d9bb11133f420d75e083,rio de janeiro,RJ,sao goncalo,RJ,"[{'product_id': '3630657a252ca88c694edfa53f2ad9f2', 'product_count': 1}]",10.0,10.0,0.0,2.0,243.500000,3.00,100.0,0.0,12,9.973333,4
2,a4bd6e9adf39b63f43dc545d3ca1f53d,rio de janeiro,RJ,sao goncalo,RJ,"[{'product_id': 'e6baba6c7819d44817a76305e082d682', 'product_count': 1}, {'product_id': 'c511bbd742df73d5b2ba4594d4901f2b', 'product_count': 1}, {'product_id': '902ef94ec6b84c1abdc30ac2877a0e95', 'product_count': 1}]",4.0,4.0,0.0,0.0,384.250000,4.50,100.0,0.0,12,9.973333,4
3,7901646fdd36a55f564ffaf2dbccaaf7,rio de janeiro,RJ,sao goncalo,RJ,"[{'product_id': '636598095d69a5718e67d2c9a3c7dde6', 'product_count': 4}, {'product_id': '6e7df7f4622d4360261995dbd5e787d0', 'product_count': 3}, {'product_id': '8374b39a15882a19ce4558f13064b55c', 'product_count': 2}, {'product_id': 'b6397895a17ce86decd60b898b459796', 'product_count': 1}, {'product_id': '5a968ab149dac747332323c42b49e30a', 'product_count': 1}, {'product_id': '57e7f9befd08f9e14bc4545abf760d0e', 'product_count': 1}, {'product_id': '03ac940e93916395ea0483161cc84d5c', 'product_count': 1}, {'product_id': 'ff1f1de0f05e0ec5b2d721e2fb425b0c', 'product_count': 1}]",22.0,22.0,0.0,3.0,607.045455,4.50,100.0,0.0,12,9.973333,4
4,8a9260f2b0340411d6d2a56bcf4f7378,contagem,MG,sao goncalo,RJ,"[{'product_id': 'b800d7bb8cd5a7093dd099a367d1dde2', 'product_count': 2}]",8.0,8.0,0.0,1.0,630.875000,4.75,100.0,0.0,3,28.963333,4


## Kesimpulan

**1. Bagaimana performa penjualan e-commerce berdasarkan lokasi geografis?**

    Hasil Analisis:

- São Paulo (SP) merupakan pasar utama dengan performa terbaik, mencatat 15.045 pesanan dengan GMV $2.107.960,17

- Rio de Janeiro (RJ) di posisi kedua dengan 6.601 pesanan dan GMV $1.111.732,21

- Belo Horizonte (MG) menempati posisi ketiga dengan 2.697 pesanan dan GMV $405.950,51

- Wilayah tenggara Brasil (SP, RJ, MG) mendominasi kontribusi penjualan

- Terdapat konsentrasi aktivitas *e-commerce* yang kuat di area metropolitan utama

**2. Bagaimana persebaran konsumen dan nilai transaksi di berbagai wilayah?**

    Hasil Analisis:

- Persebaran geografis menunjukkan pola yang jelas dengan konsentrasi tinggi di:

  * São Paulo dan sekitarnya (Guarulhos, Campinas, São Bernardo do Campo)

  * Rio de Janeiro metropolitan area

  * Minas Gerais (Belo Horizonte sebagai pusat)

  * Brasília (DF) sebagai hub wilayah tengah

- Nilai transaksi mengikuti pola distribusi populasi dan perkembangan ekonomi:

  * Kota-kota besar menunjukkan GMV tinggi sebanding dengan volume pesanan

  * Beberapa wilayah seperti Brasília menunjukkan nilai transaksi rata-rata yang lebih tinggi meskipun volume pesanan lebih rendah

  * Terdapat 4.214 lokasi unik yang tercatat melakukan transaksi di platform